# Huber–Ridge AIME (HRA) — End-to-End Reproducibility Notebook

This Colab-ready notebook reproduces the paper's experiments and generates all publication artifacts:


In [16]:
!pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable


In [17]:
!pip install "numpy<=2.2.0"

Defaulting to user installation because normal site-packages is not writeable


In [18]:
!pip install -U shap numba

Defaulting to user installation because normal site-packages is not writeable


In [19]:
# %% [setup] Environment and versions
import sys, os, subprocess, importlib, warnings
warnings.filterwarnings('ignore')

py_major, py_minor = sys.version_info[:2]
PY_STR = f"{py_major}.{py_minor}"

# ---- Helper: safe pip install ------------------------------------------------
def pip_install(*args):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", *args])
    except subprocess.CalledProcessError as e:
        print(f"[pip] install failed: {args}\n  -> {e}")

def ensure_pkg(import_name, pip_name_or_spec=None):
    try:
        __import__(import_name)
    except Exception:
        if pip_name_or_spec is None:
            pip_name_or_spec = import_name
        pip_install(pip_name_or_spec)
        __import__(import_name)

# ---- Policy: decide SHAP/Numba/Numpy strategy by Python version --------------
WANT_SHAP = True
if (py_major, py_minor) >= (3, 13):
    # On Python 3.13, NumPy==2.3 is typical; Numba is not yet compatible -> SHAP import breaks.
    WANT_SHAP = False
    print(f"[setup] Python {PY_STR} detected. SHAP will be disabled automatically "
          f"(Numba vs NumPy 2.3 incompatibility).")
else:
    # Python 3.11/3.12: pin a known-good combo for SHAP/Numba before anything else
    print(f"[setup] Python {PY_STR} detected. Pinning NumPy/Numba combo for SHAP...")
    pip_install("numpy<=2.2.0", "numba<0.61")

# ---- Core packages ------------------------------------------------------------
core_specs = [
    ('sklearn','scikit-learn'),
    ('lightgbm','lightgbm'),
    ('lime','lime'),
    ('scipy','scipy'),
    ('tqdm','tqdm'),
    ('matplotlib','matplotlib'),
    ('pandas','pandas'),
]

for imp, spec in core_specs:
    ensure_pkg(imp, spec)

# SHAP is conditional
if WANT_SHAP:
    ensure_pkg('shap', 'shap')

# ---- Imports -----------------------------------------------------------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from lime.lime_tabular import LimeTabularExplainer
from lightgbm import LGBMClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from scipy import stats

HAS_SHAP = False
if WANT_SHAP:
    try:
        import shap
        HAS_SHAP = True
    except Exception as e:
        print("[warn] SHAP unavailable after install attempt:", e)

# ---- Version printout ---------------------------------------------------------
import importlib
print('[python]', sys.version)
print('[versions]',
      'sklearn', importlib.import_module('sklearn').__version__,
      '| lightgbm', importlib.import_module('lightgbm').__version__,
#      '| lime', importlib.import_module('lime').__version__,
      '| numpy', importlib.import_module('numpy').__version__)
if HAS_SHAP:
    print('| shap', importlib.import_module('shap').__version__)
else:
    print('| shap (disabled)')

# ---- Guidance message ---------------------------------------------------------
if not HAS_SHAP and (py_major, py_minor) >= (3, 13):
    print("\n[info] SHAP を使う場合は、Python 3.11/3.12 の仮想環境で "
          "numpy<=2.2.0 と numba<0.61 を固定してください。")

[setup] Python 3.9 detected. Pinning NumPy/Numba combo for SHAP...
[python] 3.9.6 (default, Apr 30 2025, 02:07:17) 
[Clang 17.0.0 (clang-1700.0.13.5)]
[versions] sklearn 1.6.1 | lightgbm 4.6.0 | numpy 2.0.2
| shap 0.48.0


In [20]:
# %% [config]
from pathlib import Path
import random
import json

OUTDIR = Path('/Users/takafumi/Documents/Python/output/HuberRidgeAIME_results')
OUTDIR.mkdir(parents=True, exist_ok=True)

CFG = {
    # Reproducibility
    'random_state': 42,

    # Datasets and models
    'datasets': ['breast_cancer','credit_approval','har'],
    'models':   ['lgbm','svm','mlp'],

    # AIME-family solver parameters
    'huber_delta': 1.0,
    'ridge_lambda': 1e-2,
    'max_iter': 50,
    'tol': 1e-6,

    # Stress metrics
    'top_k': 10,
    'bootstrap_B': 20,
    'noise_std_frac': 0.10,
    'decoy_trials': 30,

    # LIME/SHAP compute toggles (no skipping by default)
    'compute_lime': True,
    'compute_shap': True,

    # LIME budgets (kept modest on HAR to fit Colab Pro/standard runtime)
    'lime_sample_n':   {'breast_cancer': 256, 'credit_approval': 256, 'har': 64},
    'lime_num_samples':{'breast_cancer': 3000, 'credit_approval': 3000, 'har': 800},

    # SHAP budgets
    'shap_sample_n':   {'breast_cancer': 128, 'credit_approval': 128, 'har': 32},
    'shap_bg_n':       {'breast_cancer': 100, 'credit_approval': 100, 'har': 32},
    'shap_kernel_nsamples': {'breast_cancer': 2000, 'credit_approval': 2000, 'har': 300},

    # Synthetic grid size (≈400 paired conditions by default)
    'grid_rho': np.linspace(0.0, 0.9, 10).tolist(),
    'grid_pi':  np.linspace(0.0, 0.3, 10).tolist(),
    'grid_reps': 4,

    # Figure toggles
    'save_panels': True,
}

np.random.seed(CFG['random_state'])
random.seed(CFG['random_state'])
print('[outdir]', OUTDIR.resolve())
print(json.dumps(CFG, indent=2))

[outdir] /Users/takafumi/Documents/Python/output/HuberRidgeAIME_results
{
  "random_state": 42,
  "datasets": [
    "breast_cancer",
    "credit_approval",
    "har"
  ],
  "models": [
    "lgbm",
    "svm",
    "mlp"
  ],
  "huber_delta": 1.0,
  "ridge_lambda": 0.01,
  "max_iter": 50,
  "tol": 1e-06,
  "top_k": 10,
  "bootstrap_B": 20,
  "noise_std_frac": 0.1,
  "decoy_trials": 30,
  "compute_lime": true,
  "compute_shap": true,
  "lime_sample_n": {
    "breast_cancer": 256,
    "credit_approval": 256,
    "har": 64
  },
  "lime_num_samples": {
    "breast_cancer": 3000,
    "credit_approval": 3000,
    "har": 800
  },
  "shap_sample_n": {
    "breast_cancer": 128,
    "credit_approval": 128,
    "har": 32
  },
  "shap_bg_n": {
    "breast_cancer": 100,
    "credit_approval": 100,
    "har": 32
  },
  "shap_kernel_nsamples": {
    "breast_cancer": 2000,
    "credit_approval": 2000,
    "har": 300
  },
  "grid_rho": [
    0.0,
    0.1,
    0.2,
    0.30000000000000004,
    0.4,
    0.5

In [21]:
# %% [data loading]
import io, zipfile, urllib.request, time

def _download_bytes(urls, timeout=180, retries=3, sleep=3):
    if isinstance(urls, str): urls = [urls]
    last = None
    for u in urls:
        for _ in range(retries):
            try:
                with urllib.request.urlopen(u, timeout=timeout) as r:
                    return r.read()
            except Exception as e:
                last = e; time.sleep(sleep)
    raise last if last else RuntimeError('download failed')

def load_breast_cancer_dataset():
    data = load_breast_cancer()
    X = np.asarray(data.data, dtype=float)
    y = np.asarray(data.target, dtype=int)
    feat_names = [str(x) for x in data.feature_names]
    class_names = [str(x) for x in data.target_names]
    return X, y, feat_names, class_names

def load_credit_approval():
    url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data'
    raw = pd.read_csv(url, header=None, na_values='?')
    y = raw.iloc[:, -1].map({'+':1, '-':0}).astype(int).to_numpy()
    X_df = raw.iloc[:, :-1].copy()
    cat_cols = [i for i in X_df.columns if X_df[i].dtype == object]
    num_cols = [i for i in X_df.columns if X_df[i].dtype != object]
    for c in num_cols:
        X_df[c] = X_df[c].astype(float).fillna(X_df[c].median())
    for c in cat_cols:
        X_df[c] = X_df[c].astype('category')
        X_df[c] = X_df[c].cat.add_categories(['Unknown']).fillna('Unknown')
    X_proc = pd.get_dummies(X_df, drop_first=True)
    X = X_proc.to_numpy(dtype=float)
    feat_names = X_proc.columns.astype(str).tolist()
    class_names = [str(c) for c in np.unique(y)]
    return X, y, feat_names, class_names

def load_har():
    mirrors = [
        'https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip',
        'http://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip',
    ]
    by = _download_bytes(mirrors)
    zf = zipfile.ZipFile(io.BytesIO(by))
    root = 'UCI HAR Dataset/'
    with zf.open(root + 'features.txt') as f:
        feats = pd.read_csv(f, sep=r'\s+', header=None, names=['idx','name'])
    with zf.open(root + 'train/X_train.txt') as f:
        Xtr = np.loadtxt(f).astype(float)
    with zf.open(root + 'train/y_train.txt') as f:
        ytr = np.loadtxt(f).astype(int).ravel()
    with zf.open(root + 'test/X_test.txt') as f:
        Xte = np.loadtxt(f).astype(float)
    with zf.open(root + 'test/y_test.txt') as f:
        yte = np.loadtxt(f).astype(int).ravel()
    X = np.vstack([Xtr, Xte]).astype(float)
    y = (np.hstack([ytr, yte]) - 1).astype(int)
    feat_names = feats['name'].astype(str).tolist()
    class_names = [str(c) for c in np.unique(y)]
    return X, y, feat_names, class_names

def get_dataset(name):
    if name == 'breast_cancer': return load_breast_cancer_dataset()
    if name == 'credit_approval': return load_credit_approval()
    if name == 'har': return load_har()
    raise ValueError(name)

In [22]:
# %% [models]
def make_model(kind, random_state=42):
    if kind == 'lgbm':
        return LGBMClassifier(random_state=random_state, n_estimators=200, learning_rate=0.05, n_jobs=-1, verbosity=-1)
    if kind == 'svm':
        return SVC(kernel='rbf', probability=True, random_state=random_state, C=2.0, gamma='scale')
    if kind == 'mlp':
        return MLPClassifier(hidden_layer_sizes=(128,), activation='relu', alpha=1e-4, max_iter=200, random_state=random_state)
    raise ValueError(kind)

def fit_and_get_probs(X, y, kind, random_state=42):
    scaler = StandardScaler()
    model = make_model(kind, random_state=random_state)
    if kind in ('svm','mlp'):
        pipe = Pipeline([('scaler', scaler), ('clf', model)])
    else:
        pipe = Pipeline([('clf', model)])
    pipe.fit(X, y)
    proba = pipe.predict_proba(X)
    y_pred = np.argmax(proba, axis=1)
    return pipe, proba, y_pred

In [23]:
# %% [AIME-family: Huber–Ridge IRLS and variants]
def huber_weights(r, delta):
    r = np.asarray(r, dtype=float)
    a = np.abs(r)
    w = np.ones_like(a)
    m = a > delta
    w[m] = delta / a[m]
    return np.clip(w, 1e-8, None)

def solve_wls_ridge(X, y, sample_w, lam):
    X = np.asarray(X, dtype=float); y = np.asarray(y, dtype=float); w = np.asarray(sample_w, dtype=float)
    Xw = X * w[:, None]
    A = Xw.T @ X + lam * np.eye(X.shape[1])
    b = Xw.T @ y
    try:
        return np.linalg.solve(A, b)
    except np.linalg.LinAlgError:
        return np.linalg.pinv(A) @ b

def compute_cwfi_aime(X, Y, method='AIME', delta=1.0, ridge_lambda=1e-2, max_iter=50, tol=1e-6):
    """Compute class-wise feature importance matrix V (d, C).
    Methods: AIME (OLS), RidgeAIME (ridge LS), HuberAIME (Huber IRLS), HuberRidgeAIME (Huber + ridge).
    """
    X = np.asarray(X, dtype=float); Y = np.asarray(Y, dtype=float)
    n, d = X.shape; C = Y.shape[1]
    mu = X.mean(axis=0, keepdims=True)
    sd = X.std(axis=0, keepdims=True)
    sd = np.where(sd < 1e-12, 1.0, sd)
    Xs = (X - mu) / sd

    V = np.zeros((d, C), dtype=float)
    use_huber = method.lower().startswith('huber')
    use_ridge = ('ridge' in method.lower())
    lam = ridge_lambda if use_ridge else 0.0

    for c in range(C):
        y = Y[:, c]
        if use_huber:
            w = np.ones(n, dtype=float); wcoef = np.zeros(d, dtype=float)
            for _ in range(max_iter):
                prev = wcoef.copy()
                wcoef = solve_wls_ridge(Xs, y, w, lam)
                r = y - Xs @ wcoef
                w = huber_weights(r, delta)
                if np.linalg.norm(wcoef - prev) <= tol * (np.linalg.norm(prev) + 1e-12):
                    break
        else:
            wcoef = solve_wls_ridge(Xs, y, np.ones(n), lam)
        V[:, c] = (wcoef / sd).ravel()
    return V

In [24]:
# %% [LIME/SHAP aggregation]
def cwfi_lime_signed(model, X, class_names, *, sample_n=256, num_samples=3000, kernel_width=None, random_state=0, predict_fn=None):
    """Aggregate LIME local explanations into class-wise signed mean weights.
    A custom predict_fn can be provided to be robust to decoy columns (drop-tail wrapper).
    """
    X = np.asarray(X, dtype=float)
    n, d = X.shape
    C = len(class_names)
    rng = np.random.RandomState(random_state)
    idx = rng.choice(n, size=min(sample_n, n), replace=False)
    Xs = X[idx]
    if predict_fn is None:
        predict_fn = model.predict_proba
    yp = np.argmax(predict_fn(Xs), axis=1)

    explainer = LimeTabularExplainer(
        X, feature_names=[f'f{j}' for j in range(d)], class_names=class_names,
        discretize_continuous=False, random_state=random_state,
        kernel_width=(0.75*math.sqrt(d) if kernel_width is None else kernel_width)
    )
    V = np.zeros((d, C), dtype=float)
    for i, xi in enumerate(tqdm(Xs, desc='LIME', leave=False)):
        cl = int(yp[i])
        exp = explainer.explain_instance(xi, predict_fn, num_samples=num_samples, labels=list(range(C)))
        amap = exp.as_map(); lbls = getattr(exp, 'available_labels', None)
        if callable(lbls): lbls = lbls()
        if lbls is None: lbls = list(amap.keys())
        lbl = cl if cl in lbls else int(lbls[0])
        local = dict(amap.get(lbl, []))
        for j, wj in local.items():
            jj = int(j)
            if 0 <= jj < d:
                V[jj, cl] += float(wj)
    for c in range(C):
        cnt = int(np.sum(yp == c))
        if cnt > 0: V[:, c] /= cnt
    return V

def _align_shap_outputs_to_list(sv, m, d, C):
    arr = np.asarray(sv)
    if isinstance(sv, list):
        out = []
        for c in range(C):
            A = np.asarray(sv[c if c < len(sv) else 0], dtype=float)
            if A.ndim == 2 and A.shape == (m, d): out.append(A)
            elif A.ndim == 2 and A.shape == (d, m): out.append(A.T)
            elif A.ndim == 3 and A.shape[-1] == d: out.append(A.reshape(-1, d))
            else: out.append(A.reshape(m, d))
        return out
    if arr.ndim == 2:
        if arr.shape == (m, d): return [arr]*C
        if arr.shape == (d, m): return [arr.T]*C
    if arr.ndim == 3:
        if arr.shape == (m, C, d): return [arr[:, c, :] for c in range(C)]
        if arr.shape == (m, d, C): return [arr[:, :, c] for c in range(C)]
        if arr.shape == (C, m, d): return [arr[c, :, :] for c in range(C)]
    raise ValueError(f'Unexpected SHAP shape: {arr.shape}')

def cwfi_shap_signed(model, X, C, *, m_sample=128, bg_n=100, kernel_nsamples=2000, random_state=0, base_dim=None):
    """Class-wise signed SHAP aggregation.
    If base_dim is provided and X has more columns (due to decoy augmentation),
    a predict wrapper and KernelSHAP are used so the underlying model always receives the base_dim columns.
    LightGBM uses TreeSHAP (raw output) when dimensions match.
    """
    X = np.asarray(X, dtype=float); n, d = X.shape
    rng = np.random.RandomState(random_state)
    m = int(min(m_sample, n))
    idx = rng.choice(n, size=m, replace=False)
    Xs = X[idx]

    def predict_drop_tail(A):
        A = np.asarray(A, dtype=float)
        if base_dim is not None and A.shape[1] != base_dim:
            A = A[:, :base_dim]
        return model.predict_proba(A)
    yp = np.argmax(predict_drop_tail(Xs), axis=1)

    est = model
    if hasattr(model, 'named_steps') and 'clf' in getattr(model, 'named_steps', {}):
        est = model.named_steps['clf']
    use_tree = (base_dim is None or d == base_dim) and isinstance(est, LGBMClassifier)

    if use_tree:
        expl = shap.TreeExplainer(est, model_output='raw')
        sv = expl.shap_values(Xs)
        shap_per_class = _align_shap_outputs_to_list(sv, m, d, C)
    else:
        b = int(min(bg_n, n)); bg_idx = rng.choice(n, size=b, replace=False)
        bg = X[bg_idx]
        expl = shap.KernelExplainer(predict_drop_tail, bg)
        sv = expl.shap_values(Xs, nsamples=int(kernel_nsamples))
        shap_per_class = _align_shap_outputs_to_list(sv, m, d, C)

    V = np.zeros((d, C), dtype=float)
    for c in range(C):
        A = shap_per_class[c]
        msk = (yp == c)
        V[:, c] = A[msk].mean(axis=0) if msk.any() else A.mean(axis=0)
    return V

In [25]:
# %% [stress metrics]
def _topk_indices(V, k):
    g = np.abs(V).sum(axis=1)
    k = min(k, g.size)
    idx = np.argpartition(-g, k-1)[:k]
    return set(idx.tolist())

def compute_bootstrap_stability(expl_fn, X, B, k, rs):
    rng = np.random.RandomState(rs)
    V0 = expl_fn(X); base = _topk_indices(V0, k)
    if not base: return 0.0
    n = X.shape[0]; scores = []
    for _ in range(B):
        idx = rng.choice(n, size=n, replace=True)
        Vb = expl_fn(X[idx]); test = _topk_indices(Vb, k)
        inter = len(base & test); uni = len(base | test)
        scores.append(inter/uni if uni else 0.0)
    return float(np.mean(scores))

def compute_noise_robustness(expl_fn, X, noise_frac, trials, k, rs):
    rng = np.random.RandomState(rs)
    V0 = expl_fn(X); base = _topk_indices(V0, k)
    if not base: return 0.0
    std = X.std(axis=0, keepdims=True); std = np.where(std < 1e-12, 1.0, std)
    scores = []
    for _ in range(trials):
        Xn = X + rng.normal(0.0, noise_frac*std, size=X.shape)
        Vn = expl_fn(Xn); test = _topk_indices(Vn, k)
        inter = len(base & test); uni = len(base | test)
        scores.append(inter/uni if uni else 0.0)
    return float(np.mean(scores))

def compute_decoy_resistance(expl_fn, X, trials, k, rs):
    rng = np.random.RandomState(rs)
    d0 = X.shape[1]; intrusions = 0
    for _ in range(trials):
        j = rng.randint(0, d0)
        decoy = rng.permutation(X[:, j])
        X_aug = np.column_stack([X, decoy])
        V = expl_fn(X_aug)
        top = _topk_indices(V, k)
        if d0 in top: intrusions += 1
    return float(1.0 - intrusions/trials)

In [26]:
# %% [real data runner]
def save_cwfi_csv(V, dataset, mkind, method, feat_names=None, class_names=None):
    V = np.asarray(V, dtype=float)
    d, C = V.shape
    feat_names = feat_names or [f'f{j}' for j in range(d)]
    class_names = class_names or [str(c) for c in range(C)]
    rows = []
    for j in range(d):
        for c in range(C):
            rows.append({'dataset': dataset, 'model': mkind, 'method': method,
                         'feature': feat_names[j], 'cls': class_names[c], 'weight': float(V[j, c])})
    df = pd.DataFrame(rows)
    p = OUTDIR / f'cwfi_{dataset}__{mkind}.csv'
    mode = 'a' if p.exists() else 'w'; header = not p.exists()
    df.to_csv(p, index=False, mode=mode, header=header)
    return p

def save_panel_figure(dataset, mkind, class_names):
    df = pd.read_csv(OUTDIR / f'cwfi_{dataset}__{mkind}.csv')
    methods = ['AIME','HuberAIME','RidgeAIME','HuberRidgeAIME']
    fig, axes = plt.subplots(2, 2, figsize=(10, 6), constrained_layout=True)
    for ax, meth in zip(axes.ravel(), methods):
        piv = pd.pivot_table(
            df[df['method'] == meth], index='feature', columns='cls', values='weight', aggfunc='mean', fill_value=0.0
        )
        cols = [c for c in class_names if c in piv.columns] + [c for c in piv.columns if c not in class_names]
        W = np.abs(piv[cols].values)
        im = ax.imshow(W, aspect='auto')
        ax.set_title(meth); ax.set_xlabel('class'); ax.set_ylabel('feature')
    fig.suptitle(f'{dataset} / {mkind} — class-wise signed GFI (|weight|)')
    p = OUTDIR / f'panel_{dataset}_{mkind}.png'
    plt.savefig(p, dpi=200); plt.close(fig)
    return p

def run_real():
    np.random.seed(CFG['random_state']); random.seed(CFG['random_state'])
    for ds in CFG['datasets']:
        X, y, feat_names, class_names = get_dataset(ds)
        X = np.asarray(X, dtype=float); y = np.asarray(y, dtype=int)
        C = len(np.unique(y)); d0 = X.shape[1]
        for mk in CFG['models']:
            model, Y, y_pred = fit_and_get_probs(X, y, mk, random_state=CFG['random_state'])
            # predict wrapper robust to decoys (drop extra columns)
            def predict_drop_tail(A):
                A = np.asarray(A, dtype=float)
                if A.shape[1] != d0:
                    A = A[:, :d0]
                return model.predict_proba(A)

            # AIME family
            def expl_closure(name):
                return lambda XX: compute_cwfi_aime(
                    XX, predict_drop_tail(XX), method=name,
                    delta=CFG['huber_delta'], ridge_lambda=CFG['ridge_lambda'],
                    max_iter=CFG['max_iter'], tol=CFG['tol']
                )
            for name in ['AIME','HuberAIME','RidgeAIME','HuberRidgeAIME']:
                V = expl_closure(name)(X)
                save_cwfi_csv(V, ds, mk, name, feat_names=feat_names, class_names=class_names)
                # stress
                top_k = CFG['top_k']
                stab = compute_bootstrap_stability(expl_closure(name), X, B=CFG['bootstrap_B'], k=top_k, rs=CFG['random_state'])
                nrob = compute_noise_robustness(expl_closure(name), X, noise_frac=CFG['noise_std_frac'], trials=20, k=top_k, rs=CFG['random_state'])
                dres = compute_decoy_resistance(expl_closure(name), X, trials=CFG['decoy_trials'], k=top_k, rs=CFG['random_state'])
                row = pd.DataFrame([{'dataset': ds, 'model': mk, 'method': name,
                                     'stability': stab, 'noise': nrob, 'decoy': dres}])
                out_stress = OUTDIR / f'stress_{ds}__{mk}.csv'
                mode = 'a' if out_stress.exists() else 'w'; header = not out_stress.exists()
                row.to_csv(out_stress, index=False, mode=mode, header=header)

            # Panel
            if CFG['save_panels']:
                save_panel_figure(ds, mk, class_names)

            # LIME
            if CFG['compute_lime']:
                try:
                    V_lime = cwfi_lime_signed(
                        model, X, class_names,
                        sample_n=CFG['lime_sample_n'].get(ds, 256),
                        num_samples=CFG['lime_num_samples'].get(ds, 3000),
                        kernel_width=0.75*math.sqrt(X.shape[1]),
                        random_state=CFG['random_state'],
                        predict_fn=predict_drop_tail
                    )
                    save_cwfi_csv(V_lime, ds, mk, 'LIME', feat_names=feat_names, class_names=class_names)
                    # stress for LIME
                    def expl_LIME(XX):
                        return cwfi_lime_signed(
                            model, XX, class_names,
                            sample_n=CFG['lime_sample_n'].get(ds, 256),
                            num_samples=CFG['lime_num_samples'].get(ds, 3000),
                            kernel_width=0.75*math.sqrt(XX.shape[1]),
                            random_state=CFG['random_state'], predict_fn=predict_drop_tail)
                    top_k = CFG['top_k']
                    row = pd.DataFrame([{
                        'dataset': ds, 'model': mk, 'method': 'LIME',
                        'stability': compute_bootstrap_stability(expl_LIME, X, B=8, k=top_k, rs=CFG['random_state']),
                        'noise':     compute_noise_robustness(expl_LIME, X, noise_frac=CFG['noise_std_frac'], trials=10, k=top_k, rs=CFG['random_state']),
                        'decoy':     compute_decoy_resistance(expl_LIME, X, trials=10, k=top_k, rs=CFG['random_state']),
                    }])
                    out_stress = OUTDIR / f'stress_{ds}__{mk}.csv'
                    mode = 'a' if out_stress.exists() else 'w'; header = not out_stress.exists()
                    row.to_csv(out_stress, index=False, mode=mode, header=header)
                except Exception as e:
                    print('[LIME error but continuing]', repr(e))

            # SHAP
            if CFG['compute_shap']:
                try:
                    V_shap = cwfi_shap_signed(
                        model, X, C,
                        m_sample=CFG['shap_sample_n'].get(ds, 128),
                        bg_n=CFG['shap_bg_n'].get(ds, 100),
                        kernel_nsamples=CFG['shap_kernel_nsamples'].get(ds, 2000),
                        random_state=CFG['random_state'], base_dim=d0
                    )
                    save_cwfi_csv(V_shap, ds, mk, 'SHAP', feat_names=feat_names, class_names=class_names)
                    # stress for SHAP
                    def expl_SHAP(XX):
                        return cwfi_shap_signed(
                            model, XX, C,
                            m_sample=CFG['shap_sample_n'].get(ds, 128),
                            bg_n=CFG['shap_bg_n'].get(ds, 100),
                            kernel_nsamples=CFG['shap_kernel_nsamples'].get(ds, 2000),
                            random_state=CFG['random_state'], base_dim=d0)
                    top_k = CFG['top_k']
                    row = pd.DataFrame([{
                        'dataset': ds, 'model': mk, 'method': 'SHAP',
                        'stability': compute_bootstrap_stability(expl_SHAP, X, B=5, k=top_k, rs=CFG['random_state']),
                        'noise':     compute_noise_robustness(expl_SHAP, X, noise_frac=CFG['noise_std_frac'], trials=6, k=top_k, rs=CFG['random_state']),
                        'decoy':     compute_decoy_resistance(expl_SHAP, X, trials=6, k=top_k, rs=CFG['random_state']),
                    }])
                    out_stress = OUTDIR / f'stress_{ds}__{mk}.csv'
                    mode = 'a' if out_stress.exists() else 'w'; header = not out_stress.exists()
                    row.to_csv(out_stress, index=False, mode=mode, header=header)
                except Exception as e:
                    print('[SHAP error but continuing]', repr(e))

In [27]:
# %% [synthetic control grid]
def make_correlated_X(n, d, rho, rs=0):
    rng = np.random.RandomState(rs)
    # Toeplitz covariance matrix
    idx = np.arange(d)
    cov = rho ** np.abs(np.subtract.outer(idx, idx))
    L = np.linalg.cholesky(cov + 1e-9*np.eye(d))
    Z = rng.normal(size=(n, d))
    return Z @ L.T

def contaminate_outliers(X, pi, scale=10.0, rs=0):
    if pi <= 0: return X
    rng = np.random.RandomState(rs)
    Xc = X.copy()
    n = X.shape[0]
    m = int(np.floor(pi * n))
    if m <= 0: return Xc
    idx = rng.choice(n, size=m, replace=False)
    # heavy-tailed contamination
    Xc[idx] += rng.standard_t(df=3, size=Xc[idx].shape) * scale
    return Xc

def run_synthetic_grid(n=600, d=40, C=2):
    rows = []
    rng = np.random.RandomState(CFG['random_state'])
    # True linear model to generate labels
    w_true = np.zeros(d); w_true[:6] = [2.0, 1.8, 1.6, 1.4, 1.2, 1.0]
    b_true = -0.5
    for rho in CFG['grid_rho']:
        for pi in CFG['grid_pi']:
            for rep in range(CFG['grid_reps']):
                rs = (hash((rho, pi, rep)) % (2**32-1))
                X0 = make_correlated_X(n, d, rho, rs=rs)
                X = contaminate_outliers(X0, pi, scale=10.0, rs=rs+1)
                # Probabilities from logistic model
                logit = X @ w_true + b_true
                p = 1.0 / (1.0 + np.exp(-logit))
                y = (rng.uniform(size=n) < p).astype(int)

                # Fit a black-box classifier (LightGBM is fast and strong)
                model, Y, _ = fit_and_get_probs(X, y, 'lgbm', random_state=CFG['random_state'])

                # AIME-family explainers as closures on model
                def expl(name):
                    return lambda XX: compute_cwfi_aime(
                        XX, model.predict_proba(XX), method=name,
                        delta=CFG['huber_delta'], ridge_lambda=CFG['ridge_lambda'],
                        max_iter=CFG['max_iter'], tol=CFG['tol']
                    )
                methods = ['AIME','HuberAIME','RidgeAIME','HuberRidgeAIME']
                for meth in methods:
                    V = expl(meth)(X)
                    # stress metrics (k, B chosen modest for grid speed)
                    k = 10
                    stab = compute_bootstrap_stability(expl(meth), X, B=10, k=k, rs=CFG['random_state'])
                    nrb  = compute_noise_robustness(expl(meth), X, noise_frac=0.10, trials=10, k=k, rs=CFG['random_state'])
                    dres = compute_decoy_resistance(expl(meth), X, trials=10, k=k, rs=CFG['random_state'])
                    rows.append({'rho': rho, 'pi': pi, 'rep': rep, 'method': meth,
                                 'stability': stab, 'noise': nrb, 'decoy': dres})

    df = pd.DataFrame(rows)
    df.to_csv(OUTDIR / 'grid_raw.csv', index=False)
    return df

def plot_synthetic_heatmaps(df):
    # Average over reps
    agg = df.groupby(['rho','pi','method'], as_index=False)[['stability','noise','decoy']].mean()
    methods = ['AIME','HuberAIME','RidgeAIME','HuberRidgeAIME']
    metrics = ['stability','noise','decoy']
    fig, axes = plt.subplots(len(metrics), len(methods), figsize=(4*len(methods), 3.8*len(metrics)), constrained_layout=True)
    rhos = sorted(df['rho'].unique()); pis = sorted(df['pi'].unique())
    for i, met in enumerate(metrics):
        for j, meth in enumerate(methods):
            sub = agg[agg['method'] == meth]
            # pivot with rho on x, pi on y
            M = sub.pivot(index='pi', columns='rho', values=met).values
            ax = axes[i, j]
            im = ax.imshow(M, origin='lower', aspect='auto', extent=[min(rhos), max(rhos), min(pis), max(pis)])
            ax.set_title(f'{meth} / {met}')
            ax.set_xlabel('correlation $\\rho$'); ax.set_ylabel('outlier rate $\\pi$')
    p = OUTDIR / 'synthetic_heatmaps.png'
    plt.savefig(p, dpi=200); plt.close(fig)
    return p

def plot_ablation_decoy(df):
    # HRA - HuberAIME for decoy resistance
    agg = df.groupby(['rho','pi','method'], as_index=False)['decoy'].mean()
    hra = agg[agg['method'] == 'HuberRidgeAIME'].pivot(index='pi', columns='rho', values='decoy')
    hub = agg[agg['method'] == 'HuberAIME'].pivot(index='pi', columns='rho', values='decoy')
    D = (hra - hub).values
    rhos = sorted(df['rho'].unique()); pis = sorted(df['pi'].unique())
    fig, ax = plt.subplots(1, 1, figsize=(6, 4.5), constrained_layout=True)
    im = ax.imshow(D, origin='lower', aspect='auto', extent=[min(rhos), max(rhos), min(pis), max(pis)])
    ax.set_title('Decoy resistance: HRA - HuberAIME')
    ax.set_xlabel('correlation $\\rho$'); ax.set_ylabel('outlier rate $\\pi$')
    p = OUTDIR / 'ablation_decoy_HRA_minus_HuberAIME.png'
    plt.savefig(p, dpi=200); plt.close(fig)
    return p

In [28]:
# -*- coding: utf-8 -*-
"""
HuberRidgeAIME: end-to-end, from scratch (Colab-ready)
- Datasets: breast_cancer, credit_approval, har (robust UCI HAR downloader)
- Models: LightGBM, SVM(RBF), MLP
- Explanation methods:
    * AIME family: AIME / HuberAIME / RidgeAIME / HuberRidgeAIME
    * Baselines: LIME (signed aggregation) / SHAP (Tree for LGBM, Kernel for SVM/MLP)
- Stress tests: Top-K stability (bootstrap), noise robustness (Gaussian input noise),
               decoy resistance (permuted-feature injection)
- Synthetic control grid over correlation (rho) x outlier rate (pi)
- Figures: per-condition AIME-family panels, synthetic heatmaps, HRA-HuberAIME decoy ablation
- Tables (LaTeX): overall_mean_std.tex (9 real conditions), synthetic_stats.tex (paired tests with r-effect and HL CI)
- Packaging: result.zip (everything inside /content/HuberRidgeAIME_results)
All code paths are deterministic given CFG['random_state'].
"""

# =========================
# 0) Imports and on-demand install
# =========================
import os, sys, json, math, time, io, zipfile, pathlib, warnings, importlib, itertools, random
warnings.filterwarnings("ignore")

def _ensure_pkg(import_name, pip_name=None):
    try:
        __import__(import_name)
    except Exception:
        import subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pip_name or import_name])
        __import__(import_name)

# Make sure key libs exist (Colab safe)
for imp, pipn in [
    ("numpy","numpy"),
    ("pandas","pandas"),
    ("sklearn","scikit-learn"),
    ("lightgbm","lightgbm"),
    ("shap","shap"),
    ("lime","lime"),
    ("scipy","scipy"),
    ("matplotlib","matplotlib"),
    ("tqdm","tqdm"),
]:
    _ensure_pkg(imp, pipn)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy import stats
from lime.lime_tabular import LimeTabularExplainer
from lightgbm import LGBMClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

# =========================
# 1) Global config
# =========================
OUTDIR = pathlib.Path("/Users/takafumi/Documents/Python/output/HuberRidgeAIME_results")
OUTDIR.mkdir(parents=True, exist_ok=True)

CFG = {
    # Datasets and models to run on real data
    "datasets": ["breast_cancer", "credit_approval", "har"],  # you can drop "har" if time-constrained
    "models":   ["lgbm", "svm", "mlp"],

    # AIME-family hyperparameters
    "huber_delta": 1.0,
    "ridge_lambda": 1e-2,
    "max_iter": 50,
    "tol": 1e-6,

    # Stress metrics
    "top_k": 10,
    "bootstrap_B": 20,
    "noise_std_frac": 0.10,     # Gaussian noise: std = frac * feature-wise std
    "decoy_trials": 30,

    # Compute LIME/SHAP (no skipping). To keep runtime manageable, budgets are dataset-specific.
    "compute_lime": True,
    "compute_shap": True,

    # LIME budgets
    "lime_sample_n":   {"breast_cancer": 256, "credit_approval": 256, "har": 64},
    "lime_num_samples":{"breast_cancer": 3000, "credit_approval": 3000, "har": 800},

    # SHAP budgets
    "shap_sample_n":       {"breast_cancer": 128, "credit_approval": 128, "har": 32},
    "shap_bg_n":           {"breast_cancer": 100, "credit_approval": 100, "har": 32},
    "shap_kernel_nsamples":{"breast_cancer": 2000, "credit_approval": 2000, "har": 300},

    # Synthetic grid (reduce for quick runs; increase for journal-scale)
    "synth_d": 40,           # feature dim
    "synth_n": 2000,         # samples
    "synth_C": 2,            # classes
    "synth_rhos": [-0.8,-0.5,-0.2,0.2,0.5,0.8],     # correlation grid
    "synth_pis":  [0.0,0.05,0.10,0.20,0.30],        # outlier rate grid
    "synth_reps": 8,         # replicates per cell (6*5*8=240 conditions). Increase to 16~20 for ~400.
    "synth_model": "mlp",    # model for synthetic (mlp|svm|lgbm)

    "save_panels": True,
    "save_synth_figs": True,
    "random_state": 42,
}

# Reproducibility
np.random.seed(CFG["random_state"])
random.seed(CFG["random_state"])

print("[outdir]", OUTDIR)
print("[config]", json.dumps(CFG, indent=2))


# =========================
# 2) Data loaders
# =========================
def _download_bytes(urls, timeout=180, retries=3, sleep=3) -> bytes:
    if isinstance(urls, str): urls = [urls]
    last = None
    import urllib.request, time as _t
    for u in urls:
        for _ in range(retries):
            try:
                with urllib.request.urlopen(u, timeout=timeout) as r:
                    return r.read()
            except Exception as e:
                last = e; _t.sleep(sleep)
    raise last if last else RuntimeError("download failed")

def load_breast_cancer_dataset():
    data = load_breast_cancer()
    X = np.asarray(data.data, dtype=float)
    y = np.asarray(data.target, dtype=int)
    feat_names = data.feature_names.astype(str).tolist()
    class_names = [str(c) for c in data.target_names]
    return X, y, feat_names, class_names

def load_credit_approval():
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data"
    raw = pd.read_csv(url, header=None, na_values="?")
    y = raw.iloc[:, -1].map({'+':1, '-':0}).astype(int).to_numpy()
    X_df = raw.iloc[:, :-1].copy()
    cat_cols = [i for i in X_df.columns if X_df[i].dtype == object]
    num_cols = [i for i in X_df.columns if X_df[i].dtype != object]
    for c in num_cols:
        X_df[c] = X_df[c].astype(float).fillna(X_df[c].median())
    for c in cat_cols:
        X_df[c] = X_df[c].astype("category")
        X_df[c] = X_df[c].cat.add_categories(["Unknown"]).fillna("Unknown")
    X_proc = pd.get_dummies(X_df, drop_first=True)
    X = X_proc.to_numpy(dtype=float)
    feat_names = X_proc.columns.astype(str).tolist()
    class_names = [str(c) for c in np.unique(y)]
    return X, y, feat_names, class_names

def load_har():
    mirrors = [
        "https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip",
        "http://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip",
    ]
    by = _download_bytes(mirrors)
    zf = zipfile.ZipFile(io.BytesIO(by))
    rootz = "UCI HAR Dataset/"
    with zf.open(rootz + "features.txt") as f:
        feats = pd.read_csv(f, sep=r"\s+", header=None, names=["idx","name"])
    with zf.open(rootz + "train/X_train.txt") as f:
        Xtr = np.loadtxt(f).astype(float)
    with zf.open(rootz + "train/y_train.txt") as f:
        ytr = np.loadtxt(f).astype(int).ravel()
    with zf.open(rootz + "test/X_test.txt") as f:
        Xte = np.loadtxt(f).astype(float)
    with zf.open(rootz + "test/y_test.txt") as f:
        yte = np.loadtxt(f).astype(int).ravel()
    X = np.vstack([Xtr, Xte]).astype(float)
    y = (np.hstack([ytr, yte]) - 1).astype(int)   # classes 0..5
    feat_names = feats["name"].astype(str).tolist()
    class_names = [str(c) for c in np.unique(y)]
    return X, y, feat_names, class_names


# =========================
# 3) Models
# =========================
def make_model(mkind, random_state=42):
    if mkind == "lgbm":
        return LGBMClassifier(random_state=random_state, n_estimators=200, learning_rate=0.05, verbosity=-1, n_jobs=-1)
    elif mkind == "svm":
        return SVC(kernel="rbf", probability=True, random_state=random_state, C=2.0, gamma="scale")
    elif mkind == "mlp":
        return MLPClassifier(hidden_layer_sizes=(128,), activation="relu", alpha=1e-4, max_iter=200, random_state=random_state)
    else:
        raise ValueError(mkind)

def fit_and_get_probs(X, y, mkind, rs=42):
    scaler = StandardScaler()
    model = make_model(mkind, random_state=rs)
    if mkind in ("svm","mlp"):
        pipe = Pipeline([("scaler", scaler), ("clf", model)])
    else:
        pipe = Pipeline([("clf", model)])
    pipe.fit(X, y)
    proba = pipe.predict_proba(X)
    y_pred = np.argmax(proba, axis=1)
    return pipe, proba, y_pred


# =========================
# 4) AIME-family (global operator)
# =========================
def _huber_weights(r: np.ndarray, delta: float) -> np.ndarray:
    r = np.asarray(r, dtype=float)
    a = np.abs(r)
    w = np.ones_like(a)
    mask = a > delta
    w[mask] = delta / a[mask]
    return np.clip(w, 1e-8, None)

def _solve_wls_ridge(X: np.ndarray, y: np.ndarray, sample_w: np.ndarray, lam: float) -> np.ndarray:
    X = np.asarray(X, dtype=float); y = np.asarray(y, dtype=float); sample_w = np.asarray(sample_w, dtype=float)
    Xw = X * sample_w[:, None]
    A = Xw.T @ X + lam * np.eye(X.shape[1])
    b = Xw.T @ y
    try:
        return np.linalg.solve(A, b)
    except np.linalg.LinAlgError:
        return np.linalg.pinv(A) @ b

def compute_cwfi_aime(X: np.ndarray, Y: np.ndarray, method="AIME",
                      delta=1.0, ridge_lambda=1e-2, max_iter=50, tol=1e-6) -> np.ndarray:
    """
    Compute class-wise feature importance matrix V (d, C) from inputs X (n,d) and targets Y (n,C).
    AIME:         OLS
    RidgeAIME:    Ridge-regularized LS
    HuberAIME:    Huber IRLS
    HuberRidgeAIME: Hubber IRLS + Ridge
    """
    X = np.asarray(X, dtype=float); Y = np.asarray(Y, dtype=float)
    n, d = X.shape; C = Y.shape[1]
    mu = X.mean(axis=0, keepdims=True)
    sd = X.std(axis=0, keepdims=True); sd = np.where(sd < 1e-12, 1.0, sd)
    Xs = (X - mu) / sd

    V = np.zeros((d, C), dtype=float)
    use_huber = method.lower().startswith("huber")
    use_ridge = ("ridge" in method.lower())
    lam = ridge_lambda if use_ridge else 0.0

    for c in range(C):
        y = Y[:, c]
        if use_huber:
            w = np.ones(n, dtype=float); wcoef = np.zeros(d, dtype=float)
            for _ in range(max_iter):
                prev = wcoef.copy()
                wcoef = _solve_wls_ridge(Xs, y, w, lam)
                r = y - Xs @ wcoef
                w = _huber_weights(r, delta)
                if np.linalg.norm(wcoef - prev) <= tol * (np.linalg.norm(prev) + 1e-12):
                    break
        else:
            wcoef = _solve_wls_ridge(Xs, y, np.ones(n), lam)
        V[:, c] = (wcoef / sd).ravel()
    return V


# =========================
# 5) LIME/SHAP aggregations
# =========================
def cwfi_lime_signed(model, X, class_names, *, sample_n=256, num_samples=3000,
                     kernel_width=None, random_state=0, predict_fn=None):
    """
    Signed class-wise aggregation of LIME local weights.
    predict_fn allows a wrapper that drops a decoy column (last col) when present.
    """
    X = np.asarray(X, dtype=float)
    n, d = X.shape
    C = len(class_names)
    rng = np.random.RandomState(random_state)
    idx = rng.choice(n, size=min(sample_n, n), replace=False)
    Xs = X[idx]
    if predict_fn is None:
        predict_fn = model.predict_proba
    yp = np.argmax(predict_fn(Xs), axis=1)

    explainer = LimeTabularExplainer(
        X, feature_names=[f"f{j}" for j in range(d)], class_names=class_names,
        discretize_continuous=False, random_state=random_state,
        kernel_width=(0.75*math.sqrt(d) if kernel_width is None else kernel_width)
    )
    V = np.zeros((d, C), dtype=float)
    for i, xi in enumerate(tqdm(Xs, desc="LIME", leave=False)):
        cl = int(yp[i])
        exp = explainer.explain_instance(xi, predict_fn, num_samples=num_samples, labels=list(range(C)))
        amap = exp.as_map()
        lbls = getattr(exp, "available_labels", None)
        if callable(lbls): lbls = lbls()
        if lbls is None: lbls = list(amap.keys())
        lbl = cl if cl in lbls else int(lbls[0])
        local = dict(amap.get(lbl, []))
        for j, wj in local.items():
            jj = int(j)
            if 0 <= jj < d:
                V[jj, cl] += float(wj)
    for c in range(C):
        cnt = int(np.sum(yp == c))
        if cnt > 0: V[:, c] /= cnt
    return V

def _align_shap_outputs_to_list(sv, m, d, C):
    arr = np.asarray(sv)
    if isinstance(sv, list):
        out = []
        for c in range(C):
            A = np.asarray(sv[c if c < len(sv) else 0], dtype=float)
            if A.ndim == 2 and A.shape == (m, d): out.append(A)
            elif A.ndim == 2 and A.shape == (d, m): out.append(A.T)
            elif A.ndim == 3 and A.shape[-1] == d: out.append(A.reshape(-1, d))
            else: out.append(A.reshape(m, d))
        return out
    if arr.ndim == 2:
        if arr.shape == (m, d): return [arr]*C
        if arr.shape == (d, m): return [arr.T]*C
    if arr.ndim == 3:
        if arr.shape == (m, C, d): return [arr[:, c, :] for c in range(C)]
        if arr.shape == (m, d, C): return [arr[:, :, c] for c in range(C)]
        if arr.shape == (C, m, d): return [arr[c, :, :] for c in range(C)]
    raise ValueError(f"Unexpected SHAP shape: {arr.shape}")

def cwfi_shap_signed(model, X, C, *, m_sample=128, bg_n=100, kernel_nsamples=2000,
                     random_state=0, base_dim=None):
    """
    Signed class-wise aggregation of SHAP values.
    If base_dim is set and X has more columns than base_dim (e.g., decoy), use a predict wrapper that drops the tail.
      - LightGBM: TreeExplainer(model_output='raw')
      - Others: KernelExplainer
    """
    rng = np.random.RandomState(random_state)
    X = np.asarray(X, dtype=float); n, d = X.shape
    m = int(min(m_sample, n))
    idx = rng.choice(n, size=m, replace=False)
    Xs = X[idx]

    def predict_drop_tail(A):
        A = np.asarray(A, dtype=float)
        if base_dim is not None and A.shape[1] != base_dim:
            A = A[:, :base_dim]
        return model.predict_proba(A)

    yp = np.argmax(predict_drop_tail(Xs), axis=1)

    # Resolve estimator (for Pipeline)
    est = model
    if hasattr(model, "named_steps") and "clf" in getattr(model, "named_steps", {}):
        est = model.named_steps["clf"]

    use_tree = False
    if base_dim is None or d == base_dim:
        use_tree = isinstance(est, LGBMClassifier)

    if use_tree:
        import shap as _shap
        expl = _shap.TreeExplainer(est, model_output="raw")
        sv = expl.shap_values(Xs)
        shap_per_class = _align_shap_outputs_to_list(sv, m, d, C)
    else:
        b = int(min(bg_n, n))
        bg_idx = rng.choice(n, size=b, replace=False)
        background = X[bg_idx]
        import shap as _shap
        expl = _shap.KernelExplainer(predict_drop_tail, background)
        sv = expl.shap_values(Xs, nsamples=int(kernel_nsamples))
        shap_per_class = _align_shap_outputs_to_list(sv, m, d, C)

    V = np.zeros((d, C), dtype=float)
    for c in range(C):
        A = shap_per_class[c]
        msk = (yp == c)
        V[:, c] = A[msk].mean(axis=0) if msk.any() else A.mean(axis=0)
    return V


# =========================
# 6) Stress metrics (Top-K Jaccard)
# =========================
def _topk_indices(V, k):
    g = np.abs(V).sum(axis=1)
    k = min(k, g.size)
    idx = np.argpartition(-g, k-1)[:k]
    return set(idx.tolist())

def compute_bootstrap_stability(expl_fn, X, Y_or_None, B, k, rs):
    rng = np.random.RandomState(rs)
    V0 = expl_fn(X, Y_or_None); base = _topk_indices(V0, k)
    if not base: return 0.0
    n = X.shape[0]; scores=[]
    for _ in range(B):
        idx = rng.choice(n, size=n, replace=True)
        Vb = expl_fn(X[idx], None if Y_or_None is None else Y_or_None[idx])
        test = _topk_indices(Vb, k)
        inter = len(base & test); uni = len(base | test)
        scores.append(inter/uni if uni else 0.0)
    return float(np.mean(scores))

def compute_noise_robustness(expl_fn, X, Y_or_None, noise_frac, trials, k, rs):
    rng = np.random.RandomState(rs)
    V0 = expl_fn(X, Y_or_None); base = _topk_indices(V0, k)
    if not base: return 0.0
    std = X.std(axis=0, keepdims=True); std = np.where(std < 1e-12, 1.0, std)
    scores=[]
    for _ in range(trials):
        Xn = X + rng.normal(0.0, noise_frac*std, size=X.shape)
        Vn = expl_fn(Xn, Y_or_None)
        test = _topk_indices(Vn, k)
        inter = len(base & test); uni = len(base | test)
        scores.append(inter/uni if uni else 0.0)
    return float(np.mean(scores))

def compute_decoy_resistance(expl_fn, X, Y_or_None, trials, k, rs):
    rng = np.random.RandomState(rs)
    d0 = X.shape[1]; intrusions=0
    for _ in range(trials):
        j = rng.randint(0, d0)
        decoy = rng.permutation(X[:, j])
        X_aug = np.column_stack([X, decoy])  # d0+1 cols
        V = expl_fn(X_aug, Y_or_None)        # expl_fn must handle decoy internally if needed
        top = _topk_indices(V, k)
        if d0 in top: intrusions += 1        # tail index is the injected decoy
    return float(1.0 - intrusions/trials)


# =========================
# 7) Save utilities & panels
# =========================
def save_cwfi_csv(V: np.ndarray, dataset: str, mkind: str, method: str,
                  feat_names=None, class_names=None) -> pathlib.Path:
    V = np.asarray(V, dtype=float)
    d, C = V.shape
    feat_names = feat_names or [f"f{j}" for j in range(d)]
    class_names = class_names or [str(c) for c in range(C)]
    rows = []
    for j in range(d):
        for c in range(C):
            rows.append({"dataset": dataset, "model": mkind, "method": method,
                         "feature": feat_names[j], "cls": class_names[c], "weight": float(V[j, c])})
    df = pd.DataFrame(rows)
    p = OUTDIR / f"cwfi_{dataset}__{mkind}.csv"
    mode = "a" if p.exists() else "w"
    header = not p.exists()
    df.to_csv(p, index=False, mode=mode, header=header)
    return p

def save_panel_figure(dataset, mkind, class_names):
    df = pd.read_csv(OUTDIR / f"cwfi_{dataset}__{mkind}.csv")
    methods = ["AIME","HuberAIME","RidgeAIME","HuberRidgeAIME", "LINE", "SHAP"]
    fig, axes = plt.subplots(2, 2, figsize=(10, 6), constrained_layout=True)
    for ax, meth in zip(axes.ravel(), methods):
        piv = pd.pivot_table(df[df["method"] == meth],
                             index="feature", columns="cls", values="weight",
                             aggfunc="mean", fill_value=0.0)
        cols = [c for c in class_names if c in piv.columns] + [c for c in piv.columns if c not in class_names]
        W = np.abs(piv[cols].values)
        ax.imshow(W, aspect="auto")
        ax.set_title(meth); ax.set_xlabel("class"); ax.set_ylabel("feature")
    fig.suptitle(f"{dataset} / {mkind} — class-wise signed GFI (|weight|)")
    p = OUTDIR / f"panel_{dataset}_{mkind}.png"
    plt.savefig(p, dpi=200); plt.close(fig)
    print("  [panel saved]", p)


# =========================
# 8) Runner for real datasets
# =========================
def run_for_dataset(dataset):
    # Load data
    if dataset == "breast_cancer":
        X, y, feat_names, class_names = load_breast_cancer_dataset()
    elif dataset == "credit_approval":
        X, y, feat_names, class_names = load_credit_approval()
    elif dataset == "har":
        X, y, feat_names, class_names = load_har()
    else:
        raise ValueError(dataset)

    X = np.asarray(X, dtype=float); y = np.asarray(y, dtype=int)
    C = len(np.unique(y))

    for mkind in CFG["models"]:
        print(f"[{dataset}] model={mkind}")
        model, Y, y_pred = fit_and_get_probs(X, y, mkind, rs=CFG["random_state"])

        # AIME-family
        def make_expl(name):
            return lambda XX, YY: compute_cwfi_aime(
                XX, YY, method=name,
                delta=CFG["huber_delta"], ridge_lambda=CFG["ridge_lambda"],
                max_iter=CFG["max_iter"], tol=CFG["tol"]
            )

        methods = ["AIME","HuberAIME","RidgeAIME","HuberRidgeAIME", "LINE", "SHAP"]
        for name in methods:
            expl = make_expl(name)
            V = expl(X, Y)
            save_cwfi_csv(V, dataset, mkind, name, feat_names=feat_names, class_names=class_names)

            # Stress tests for AIME-family (fast)
            top_k = CFG["top_k"]
            stab = compute_bootstrap_stability(expl, X, Y, B=CFG["bootstrap_B"], k=top_k, rs=CFG["random_state"])
            nrob = compute_noise_robustness(expl, X, Y, noise_frac=CFG["noise_std_frac"], trials=20, k=top_k, rs=CFG["random_state"])
            dres = compute_decoy_resistance(expl, X, Y, trials=CFG["decoy_trials"], k=top_k, rs=CFG["random_state"])
            df_row = pd.DataFrame([{"dataset": dataset, "model": mkind, "method": name,
                                    "stability": stab, "noise": nrob, "decoy": dres}])
            p = OUTDIR / f"stress_{dataset}__{mkind}.csv"
            mode = "a" if p.exists() else "w"; header = not p.exists()
            df_row.to_csv(p, index=False, mode=mode, header=header)

        # Optional panels
        if CFG["save_panels"]:
            save_panel_figure(dataset, mkind, class_names)

        # LIME
        if CFG.get("compute_lime", False):
            try:
                d0 = X.shape[1]
                def predict_drop_tail(A):
                    A = np.asarray(A, dtype=float)
                    if A.shape[1] != d0: A = A[:, :d0]
                    return model.predict_proba(A)
                V_lime = cwfi_lime_signed(
                    model, X, class_names,
                    sample_n=CFG["lime_sample_n"].get(dataset, 256),
                    num_samples=CFG["lime_num_samples"].get(dataset, 3000),
                    kernel_width=0.75*math.sqrt(X.shape[1]),
                    random_state=CFG["random_state"],
                    predict_fn=predict_drop_tail
                )
                save_cwfi_csv(V_lime, dataset, mkind, "LIME", feat_names=feat_names, class_names=class_names)

                # Stress for LIME (lighter budgets)
                def expl_LIME(XX, _Yunused):  # Y not used
                    return cwfi_lime_signed(model, XX, class_names,
                                             sample_n=CFG["lime_sample_n"].get(dataset, 256),
                                             num_samples=CFG["lime_num_samples"].get(dataset, 3000),
                                             kernel_width=0.75*math.sqrt(XX.shape[1]),
                                             random_state=CFG["random_state"],
                                             predict_fn=predict_drop_tail)
                top_k = CFG["top_k"]
                stab = compute_bootstrap_stability(expl_LIME, X, None, B=8, k=top_k, rs=CFG["random_state"])
                nrob = compute_noise_robustness(expl_LIME, X, None, noise_frac=CFG["noise_std_frac"], trials=10, k=top_k, rs=CFG["random_state"])
                dres = compute_decoy_resistance(expl_LIME, X, None, trials=10, k=top_k, rs=CFG["random_state"])
                p = OUTDIR / f"stress_{dataset}__{mkind}.csv"
                pd.DataFrame([{"dataset": dataset, "model": mkind, "method": "LIME",
                               "stability": stab, "noise": nrob, "decoy": dres}]
                            ).to_csv(p, index=False, mode=("a" if os.path.exists(p) else "w"), header=not os.path.exists(p))
            except Exception as e:
                print("  [LIME error but continuing]", repr(e))

        # SHAP
        if CFG.get("compute_shap", False):
            try:
                d0 = X.shape[1]
                def expl_SHAP(XX, _Yunused):
                    return cwfi_shap_signed(
                        model, XX, C,
                        m_sample=CFG["shap_sample_n"].get(dataset, 128),
                        bg_n=CFG["shap_bg_n"].get(dataset, 100),
                        kernel_nsamples=CFG["shap_kernel_nsamples"].get(dataset, 2000),
                        random_state=CFG["random_state"],
                        base_dim=d0
                    )
                V_shap = expl_SHAP(X, None)
                save_cwfi_csv(V_shap, dataset, mkind, "SHAP", feat_names=feat_names, class_names=class_names)

                # Stress for SHAP (lighter budgets)
                top_k = CFG["top_k"]
                stab = compute_bootstrap_stability(expl_SHAP, X, None, B=5, k=top_k, rs=CFG["random_state"])
                nrob = compute_noise_robustness(expl_SHAP, X, None, noise_frac=CFG["noise_std_frac"], trials=6, k=top_k, rs=CFG["random_state"])
                dres = compute_decoy_resistance(expl_SHAP, X, None, trials=6, k=top_k, rs=CFG["random_state"])
                p = OUTDIR / f"stress_{dataset}__{mkind}.csv"
                pd.DataFrame([{"dataset": dataset, "model": mkind, "method": "SHAP",
                               "stability": stab, "noise": nrob, "decoy": dres}]
                            ).to_csv(p, index=False, mode=("a" if os.path.exists(p) else "w"), header=not os.path.exists(p))
            except Exception as e:
                print("  [SHAP error but continuing]", repr(e))


# =========================
# 9) Synthetic control grid (rho x pi)
# =========================
def _equicorr_cov(d, rho):
    # Equicorrelation covariance: diag=1, off-diag=rho
    S = np.full((d, d), rho, dtype=float)
    np.fill_diagonal(S, 1.0)
    return S

def _synth_dataset(d, n, C, rho, pi, rs=0):
    rng = np.random.RandomState(rs)
    cov = _equicorr_cov(d, rho)
    # Make cov PSD even when rho close to +/-1
    # Slight ridge to ensure numerically well-conditioned
    cov = cov + 1e-6*np.eye(d)
    X = rng.multivariate_normal(mean=np.zeros(d), cov=cov, size=n)
    # Ground-truth linear score + logistic link
    w = rng.normal(0, 1, size=(d,))
    score = X @ w
    p = 1.0 / (1.0 + np.exp(-score))
    y = (p > np.median(p)).astype(int)
    # Introduce outliers: contaminate pi fraction by amplifying features (heavy tails)
    m = int(round(pi * n))
    if m > 0:
        idx = rng.choice(n, size=m, replace=False)
        X[idx] += rng.standard_t(df=2, size=(m, d)) * 5.0  # heavy-tailed bursts
    # Return
    return X.astype(float), y.astype(int)

def run_synthetic_grid():
    rows = []
    for rho in CFG["synth_rhos"]:
        for pi in CFG["synth_pis"]:
            for rep in range(CFG["synth_reps"]):
                rs = (CFG["random_state"] + 1000*rep + int((rho+1)*100) + int(pi*1000))
                X, y = _synth_dataset(CFG["synth_d"], CFG["synth_n"], CFG["synth_C"], rho, pi, rs=rs)
                model, Y, _ = fit_and_get_probs(X, y, CFG["synth_model"], rs=rs)

                def expl(name):
                    return lambda XX, YY: compute_cwfi_aime(XX, YY, method=name,
                                                            delta=CFG["huber_delta"],
                                                            ridge_lambda=CFG["ridge_lambda"],
                                                            max_iter=CFG["max_iter"], tol=CFG["tol"])
                for name in ["AIME","HuberAIME","RidgeAIME","HuberRidgeAIME", "LINE", "SHAP"]:
                    ef = expl(name)
                    # Compute stress metrics (use smaller budgets for speed)
                    stab = compute_bootstrap_stability(ef, X, Y, B=10, k=CFG["top_k"], rs=rs)
                    nrob = compute_noise_robustness(ef, X, Y, noise_frac=CFG["noise_std_frac"], trials=8, k=CFG["top_k"], rs=rs)
                    dres = compute_decoy_resistance(ef, X, Y, trials=12, k=CFG["top_k"], rs=rs)
                    rows.append({"rho": rho, "pi": pi, "rep": rep, "method": name,
                                 "stability": stab, "noise": nrob, "decoy": dres})
    df = pd.DataFrame(rows)
    df.to_csv(OUTDIR / "grid_raw.csv", index=False)
    return df


# =========================
# 10) Tables (LaTeX) for real data and synthetic grid
# =========================
def build_overall_mean_std():
    frames = []
    for p in OUTDIR.glob('stress_*.csv'):
        try:
            frames.append(pd.read_csv(p))
        except Exception:
            pass
    if not frames:
        raise FileNotFoundError('No stress_*.csv found under: ' + str(OUTDIR))
    allst = pd.concat(frames, axis=0, ignore_index=True)
    per_cond = allst.groupby(['dataset','model','method'], as_index=False)[['stability','noise','decoy']].mean()
    overall = per_cond.groupby('method')[['stability','noise','decoy']].agg(['mean','std']).reset_index()
    order = ['AIME','HuberAIME','RidgeAIME','HuberRidgeAIME','LIME','SHAP']
    overall['order'] = overall['method'].apply(lambda m: order.index(m) if m in order else 999)
    overall = overall.sort_values('order').drop(columns='order')
    return overall

def fmt_pm(mu, sd, nd=3):
    return f"{mu:.{nd}f} $\\pm$ {sd:.{nd}f}"

def tex_overall_mean_std(overall: pd.DataFrame) -> str:
    lines = []
    lines += [
        "\\begin{table}[t]",
        "\\centering",
        "\\caption{Overall averages over 9 conditions (3 datasets $\\times$ 3 models). Mean$\\pm$std.}",
        "\\label{tab:overall_mean_std}",
        "\\small",
        "\\setlength{\\tabcolsep}{6pt}",
        "\\begin{tabular}{lccc}",
        "\\toprule",
        "\\textbf{Method} & \\textbf{Stability} & \\textbf{Noise Robust.} & \\textbf{Decoy Resist.} \\\\",
        "\\midrule",
    ]
    for _, row in overall.iterrows():
        s_mu, s_sd = row[('stability','mean')], row[('stability','std')]
        n_mu, n_sd = row[('noise','mean')], row[('noise','std')]
        d_mu, d_sd = row[('decoy','mean')], row[('decoy','std')]
        lines.append(f"{row['method']} & {fmt_pm(s_mu,s_sd)} & {fmt_pm(n_mu,n_sd)} & {fmt_pm(d_mu,d_sd)} \\\\")
    lines += ["\\bottomrule","\\end{tabular}","\\end{table}"]
    return "\n".join(lines)

def hl_and_ci(diff, alpha=0.05):
    # Hodges–Lehmann estimator for paired diff: median of Walsh averages (d_i + d_j)/2
    diff = np.asarray(diff, dtype=float)
    n = diff.size
    walsh = []
    for i in range(n):
        for j in range(i, n):
            walsh.append(0.5*(diff[i] + diff[j]))
    walsh = np.sort(np.asarray(walsh))
    hl = np.median(walsh)
    lo = np.quantile(walsh, alpha/2.0)
    hi = np.quantile(walsh, 1.0 - alpha/2.0)
    return float(hl), float(lo), float(hi)

def r_from_t(t_stat, n):
    if n is None or n <= 1 or t_stat is None or np.isnan(t_stat): return np.nan
    df = n - 1
    return float(t_stat / math.sqrt(t_stat*t_stat + df))

def r_from_wilcoxon_p(p_one_sided, n):
    if p_one_sided is None or n is None or n <= 0: return np.nan
    if p_one_sided <= 0: return float('inf')
    z = stats.norm.isf(p_one_sided)
    return float(z / math.sqrt(n))

def holm_bonferroni(p_values):
    p = np.asarray(p_values, dtype=float)
    m = len(p)
    order = np.argsort(p)
    adj = np.empty(m, dtype=float)
    for k, idx in enumerate(order):
        adj[idx] = (m - k) * p[idx]
    # step-down monotone
    rev = adj[order[::-1]]
    rev = np.maximum.accumulate(rev)
    adj2 = rev[::-1]
    out = np.empty_like(p)
    out[order] = np.minimum(1.0, adj2)
    return out

def paired_tests_on_grid(hra_name='HuberRidgeAIME', baselines=('AIME','HuberAIME','RidgeAIME','LINE', 'SHAP')):
    grid_path = OUTDIR / 'grid_raw.csv'
    if not grid_path.exists():
        raise FileNotFoundError('grid_raw.csv not found, run the synthetic grid first.')
    df_grid = pd.read_csv(grid_path)
    metrics = ['decoy','noise','stability']
    rows = []
    for met in metrics:
        tmp_rows = []
        p_t_list, p_w_list = [], []
        for base in baselines:
            sub = df_grid[df_grid['method'].isin([hra_name, base])]
            pv = sub.pivot_table(index=['rho','pi','rep'], columns='method', values=met, aggfunc='mean').dropna()
            if hra_name not in pv.columns or base not in pv.columns:
                continue
            hra = pv[hra_name].to_numpy(); bse = pv[base].to_numpy()
            dif = hra - bse; n = dif.size
            mean_hra = float(np.mean(hra)); mean_base=float(np.mean(bse)); mean_diff=float(np.mean(dif))
            sd_diff = float(np.std(dif, ddof=1)) if n>1 else np.nan
            cohen_d = float(mean_diff/sd_diff) if (n>1 and sd_diff>0) else np.nan
            frac_win = float(np.mean(dif>0))
            try:
                t_stat, p_t = stats.ttest_rel(hra, bse, alternative='greater')
            except TypeError:
                t_stat, p2 = stats.ttest_rel(hra, bse)
                p_t = p2/2.0 if t_stat>0 else 1.0 - p2/2.0
            try:
                w_stat, p_w = stats.wilcoxon(hra, bse, alternative='greater', zero_method='pratt')
            except Exception:
                w_stat, p_w = (np.nan, np.nan)
            hl, lo, hi = hl_and_ci(dif, alpha=0.05)
            tmp_rows.append(dict(metric=met, baseline=base, n=n,
                                 mean_HRA=mean_hra, mean_base=mean_base, mean_diff=mean_diff,
                                 cohen_d=cohen_d, frac_win=frac_win,
                                 t_stat=float(t_stat), p_t=float(p_t),
                                 w_stat=(float(w_stat) if not np.isnan(w_stat) else np.nan),
                                 p_wilcoxon=(float(p_w) if p_w==p_w else np.nan),
                                 r_t=r_from_t(t_stat, n), r_w=r_from_wilcoxon_p(p_w, n),
                                 hl=hl, ci_lo=lo, ci_hi=hi))
            p_t_list.append(float(p_t))
            p_w_list.append(float(p_w) if p_w==p_w else 1.0)
        if tmp_rows:
            adj_t = holm_bonferroni(p_t_list)
            adj_w = holm_bonferroni(p_w_list)
            for i, row in enumerate(tmp_rows):
                row['p_t_holm'] = float(adj_t[i])
                row['p_wilcoxon_holm'] = float(adj_w[i])
                rows.append(row)
    return pd.DataFrame(rows)

def fmt_p(p):
    if p is None or (isinstance(p, float) and (np.isnan(p) or np.isinf(p))):
        return '--'
    if p < 1e-6:
        e = int(math.floor(math.log10(p)))
        m = p / (10**e)
        return f"{m:.2f}\\times 10^{{{e}}}"
    return f"{p:.3g}"

def tex_paired_table(df: pd.DataFrame) -> str:
    df = df.copy()
    metric_order = {'decoy':0, 'noise':1, 'stability':2}
    base_order   = {'AIME':0, 'HuberAIME':1, 'RidgeAIME':2}
    df['m'] = df['metric'].map(metric_order).fillna(9)
    df['b'] = df['baseline'].map(base_order).fillna(9)
    df = df.sort_values(['m','b']).drop(columns=['m','b'])
    lines = []
    lines += [
        "\\begin{table*}[t]",
        "\\centering",
        "\\small",
        "\\setlength{\\tabcolsep}{5pt}",
        "\\caption{\\textbf{Paired tests on synthetic grid (one-sided, HRA $>$ baseline)} with exact statistics, Holm--Bonferroni correction, r-type effect sizes, and Hodges--Lehmann shifts.}",
        "\\label{tab:synthetic_paired}",
        "\\begin{tabular}{llr rrr rrr rrr rrr}",
        "\\toprule",
        "Metric & Baseline & $n$ & Mean(HRA) & Mean(Base) & Diff & HL~(95\\%~CI) & $t$ & $p_t$ & $p_t^{\\mathrm{Holm}}$ & $r_t$ & $W$ & $p_{\\mathrm{Wilc}}$ & $p_{\\mathrm{Wilc}}^{\\mathrm{Holm}}$ & $r_{\\mathrm{Wilc}}$ \\\\",
        "\\midrule",
    ]
    for _, r in df.iterrows():
        lines.append(
            f"{r['metric']} & {r['baseline']} & {int(r['n'])} & "
            f"{r['mean_HRA']:.4f} & {r['mean_base']:.4f} & {r['mean_diff']:+.4f} & "
            f"{r['hl']:.4f}~([{r['ci_lo']:.4f},{r['ci_hi']:.4f}]) & "
            f"{r['t_stat']:+.3f} & {fmt_p(r['p_t'])} & {fmt_p(r['p_t_holm'])} & {r['r_t']:+.3f} & "
            f"{(r['w_stat'] if not np.isnan(r['w_stat']) else 0.0):.1f} & {fmt_p(r['p_wilcoxon'])} & {fmt_p(r['p_wilcoxon_holm'])} & {r['r_w']:+.3f} \\\\"
        )
    lines += ["\\bottomrule","\\end{tabular}","\\end{table*}"]
    return "\n".join(lines)

def save_tables_and_figs():
    tables_dir = OUTDIR / 'tables'; tables_dir.mkdir(exist_ok=True)
    # 1) overall
    overall = build_overall_mean_std()
    with open(tables_dir / 'overall_mean_std.tex', 'w') as f:
        f.write(tex_overall_mean_std(overall))
    # 2) paired on synthetic
    paired = paired_tests_on_grid()
    paired.to_csv(tables_dir / 'synthetic_stats_raw.csv', index=False)
    with open(tables_dir / 'synthetic_stats.tex', 'w') as f:
        f.write(tex_paired_table(paired))
    return tables_dir


# =========================
# 11) Synthetic heatmaps & ablation figures
# =========================
def draw_synthetic_heatmaps():
    p = OUTDIR / "grid_raw.csv"
    if not p.exists():
        print("[synthetic] grid_raw.csv not found; skip heatmaps")
        return
    df = pd.read_csv(p)
    metrics = ["stability","noise","decoy"]
    methods = ["AIME","HuberAIME","RidgeAIME","HuberRidgeAIME", "LINE", "SHAP"]
    R = len(df["rep"].unique())
    # Mean over reps
    agg = (df.groupby(["rho","pi","method"])[metrics]
             .mean().reset_index())

    # Big panel: 3 rows (metrics) x 4 cols (methods)
    rh = sorted(df["rho"].unique()); ph = sorted(df["pi"].unique())
    fig, axes = plt.subplots(len(metrics), len(methods), figsize=(4*len(methods), 3*len(metrics)), constrained_layout=True)
    for i, met in enumerate(metrics):
        for j, meth in enumerate(methods):
            sub = agg[agg["method"]==meth]
            pivot = sub.pivot(index="rho", columns="pi", values=met).loc[rh, ph]
            ax = axes[i, j]
            im = ax.imshow(pivot.values, aspect="auto", origin="lower")
            ax.set_title(f"{meth} — {met}")
            ax.set_xticks(range(len(ph))); ax.set_xticklabels([f"{v:.2f}" for v in ph], rotation=45)
            ax.set_yticks(range(len(rh))); ax.set_yticklabels([f"{v:.2f}" for v in rh])
            ax.set_xlabel("pi (outlier rate)"); ax.set_ylabel("rho (correlation)")
    fig.suptitle("Synthetic grid: mean scores over reps")
    out = OUTDIR / "synthetic_heatmaps.png"
    plt.savefig(out, dpi=200); plt.close(fig)
    print("[saved]", out)

    # Ablation: decoy (HRA - HuberAIME)
    try:
        subH = agg[agg["method"]=="HuberAIME"].pivot(index="rho", columns="pi", values="decoy").loc[rh, ph]
        subR = agg[agg["method"]=="HuberRidgeAIME"].pivot(index="rho", columns="pi", values="decoy").loc[rh, ph]
        D = subR.values - subH.values
        fig2, ax2 = plt.subplots(1,1, figsize=(6,4), constrained_layout=True)
        im = ax2.imshow(D, aspect="auto", origin="lower")
        ax2.set_title("Decoy resistance: HRA - HuberAIME")
        ax2.set_xticks(range(len(ph))); ax2.set_xticklabels([f"{v:.2f}" for v in ph], rotation=45)
        ax2.set_yticks(range(len(rh))); ax2.set_yticklabels([f"{v:.2f}" for v in rh])
        ax2.set_xlabel("pi (outlier rate)"); ax2.set_ylabel("rho (correlation)")
        out2 = OUTDIR / "ablation_decoy_HRA_minus_HuberAIME.png"
        plt.savefig(out2, dpi=200); plt.close(fig2)
        print("[saved]", out2)
    except Exception as e:
        print("[warn] ablation figure not created:", repr(e))


# =========================
# 12) Aggregate overall table preview (optional) & ZIP
# =========================
def aggregate_and_zip():
    # Overall preview CSVs
    rows = []
    for p in OUTDIR.glob("stress_*.csv"):
        try: rows.append(pd.read_csv(p))
        except Exception: pass
    if rows:
        allst = pd.concat(rows, axis=0, ignore_index=True)
        overall = allst.groupby("method")[["stability","noise","decoy"]].agg(['mean','std'])
        tdir = OUTDIR / "tables"; tdir.mkdir(exist_ok=True)
        overall.to_csv(tdir / "overall_mean_std_preview.csv")
        print("[tables] saved:", tdir / "overall_mean_std_preview.csv")
    else:
        print("[tables] no stress_* CSVs found")

    # ZIP everything
    zpath = OUTDIR.parent / "result.zip"
    with zipfile.ZipFile(zpath, "w", zipfile.ZIP_DEFLATED) as zf:
        for root, _, files in os.walk(OUTDIR):
            for f in files:
                fp = os.path.join(root, f)
                arc = os.path.relpath(fp, start=OUTDIR)
                zf.write(fp, arcname=arc)
    print("[zip] created:", zpath)

    # Trigger download on Colab if available
    try:
        from google.colab import files
        files.download(str(zpath))
    except Exception:
        print("[info] Not in Colab, manual download from:", zpath)


# =========================
# 13) Main execution
# =========================
t0 = time.time()
print("[start] Real-data runs on", CFG["datasets"])
for ds in CFG["datasets"]:
    print(f"[{ds}] loading and processing ...")
    run_for_dataset(ds)
print(f"[done real] {time.time()-t0:.1f}s")

# Synthetic control grid
print("[synthetic] running rho x pi grid ...")
df_grid = run_synthetic_grid()

# Tables and figures
print("[tables] exporting LaTeX tables ...")
tables_dir = save_tables_and_figs()
print("[tables dir]", tables_dir)

if CFG["save_synth_figs"]:
    print("[figures] drawing synthetic heatmaps/ablation ...")
    draw_synthetic_heatmaps()

# ZIP
aggregate_and_zip()
print("[ALL DONE]")

[outdir] /Users/takafumi/Documents/Python/output/HuberRidgeAIME_results
[config] {
  "datasets": [
    "breast_cancer",
    "credit_approval",
    "har"
  ],
  "models": [
    "lgbm",
    "svm",
    "mlp"
  ],
  "huber_delta": 1.0,
  "ridge_lambda": 0.01,
  "max_iter": 50,
  "tol": 1e-06,
  "top_k": 10,
  "bootstrap_B": 20,
  "noise_std_frac": 0.1,
  "decoy_trials": 30,
  "compute_lime": true,
  "compute_shap": true,
  "lime_sample_n": {
    "breast_cancer": 256,
    "credit_approval": 256,
    "har": 64
  },
  "lime_num_samples": {
    "breast_cancer": 3000,
    "credit_approval": 3000,
    "har": 800
  },
  "shap_sample_n": {
    "breast_cancer": 128,
    "credit_approval": 128,
    "har": 32
  },
  "shap_bg_n": {
    "breast_cancer": 100,
    "credit_approval": 100,
    "har": 32
  },
  "shap_kernel_nsamples": {
    "breast_cancer": 2000,
    "credit_approval": 2000,
    "har": 300
  },
  "synth_d": 40,
  "synth_n": 2000,
  "synth_C": 2,
  "synth_rhos": [
    -0.8,
    -0.5,
    -0.

100%|██████████| 128/128 [00:32<00:00,  3.91it/s]


[breast_cancer] model=svm
  [panel saved] /Users/takafumi/Documents/Python/output/HuberRidgeAIME_results/panel_breast_cancer_svm.png


100%|██████████| 128/128 [01:45<00:00,  1.21it/s]


[breast_cancer] model=mlp
  [panel saved] /Users/takafumi/Documents/Python/output/HuberRidgeAIME_results/panel_breast_cancer_mlp.png


...

[har] model=svm
  [panel saved] /Users/takafumi/Documents/Python/output/HuberRidgeAIME_results/panel_har_svm.png


100%|██████████| 32/32 [02:13<00:00,  4.18s/it]


[har] model=mlp
  [panel saved] /Users/takafumi/Documents/Python/output/HuberRidgeAIME_results/panel_har_mlp.png


100%|██████████| 32/32 [00:01<00:00, 19.52it/s]


[done real] 17013.6s
[synthetic] running rho x pi grid ...
[tables] exporting LaTeX tables ...
[tables dir] /Users/takafumi/Documents/Python/output/HuberRidgeAIME_results/tables
[figures] drawing synthetic heatmaps/ablation ...
[saved] /Users/takafumi/Documents/Python/output/HuberRidgeAIME_results/synthetic_heatmaps.png
[saved] /Users/takafumi/Documents/Python/output/HuberRidgeAIME_results/ablation_decoy_HRA_minus_HuberAIME.png
[tables] saved: /Users/takafumi/Documents/Python/output/HuberRidgeAIME_results/tables/overall_mean_std_preview.csv
[zip] created: /Users/takafumi/Documents/Python/output/result.zip
[info] Not in Colab, manual download from: /Users/takafumi/Documents/Python/output/result.zip
[ALL DONE]


In [29]:
# %% [zip packaging]
import zipfile
def zip_results():
    zpath = OUTDIR.parent / 'result.zip'
    with zipfile.ZipFile(zpath, 'w', zipfile.ZIP_DEFLATED) as zf:
        for root, _, files in os.walk(OUTDIR):
            for f in files:
                fp = os.path.join(root, f)
                arc = os.path.relpath(fp, start=OUTDIR)
                zf.write(fp, arcname=arc)
    print('[zip] created:', zpath)
    return zpath

In [30]:
# %% Build TeX tables & panels from result.zip
import zipfile, io, math, os, textwrap
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
from scipy import stats
from scipy.stats import norm

ZIP = Path(OUTDIR.parent/'result.zip')
OUT = Path(OUTDIR.parent/'produce_out'); OUT.mkdir(exist_ok=True)
z = zipfile.ZipFile(ZIP, 'r')

def csv_in(name):
    with z.open(name) as f: return pd.read_csv(f)

# ---------- Load stress & compute overall mean±std ----------
method_order = ['AIME','HuberAIME','RidgeAIME','HuberRidgeAIME','LIME','SHAP']
stress_files = [n for n in z.namelist() if n.startswith('stress_') and n.endswith('.csv')]
stress = pd.concat([csv_in(n) for n in stress_files], ignore_index=True)
stress['method'] = pd.Categorical(stress['method'], categories=method_order, ordered=True)

def meanstd(s): return f"{s.mean():.3f} $\\pm$ {s.std(ddof=0):.3f}"
overall = (stress.groupby('method', observed=True)[['stability','noise','decoy']]
           .agg(meanstd).reset_index())

tex = r"""\begin{table}[t]
\centering
\caption{Overall averages over 9 conditions (3 datasets $\times$ 3 models). Values are mean$\pm$std.}
\label{tab:overall_mean_std}
\small
\setlength{\tabcolsep}{6pt}
\begin{tabular}{lccc}
\toprule
\textbf{Method} & \textbf{Stability} & \textbf{Noise Robust.} & \textbf{Decoy Resist.} \\
\midrule
"""
for _, r in overall.sort_values('method').iterrows():
    tex += f"\n{r['method']} & {r['stability']} & {r['noise']} & {r['decoy']} \\\\"
tex += r"""
\bottomrule
\end{tabular}
\end{table}
"""
(OUT/'overall_mean_std_clean.tex').write_text(tex, encoding='utf-8')

# ---------- Per-condition means with bold best (ties allowed) ----------
def per_cond(metric):
    pivot = (stress.pivot_table(index=['dataset','model'], columns='method',
                                values=metric, aggfunc='mean')
             .reindex(columns=method_order))
    sty = pivot.copy()
    for i in range(sty.shape[0]):
        row = sty.iloc[i].astype(float)
        mx = np.nanmax(row.values)
        for j, c in enumerate(sty.columns):
            v = row[c]
            sty.iloc[i, j] = f"\\textbf{{{v:.3f}}}" if abs(v-mx)<1e-12 else f"{v:.3f}"
    hdr = (r"""\begin{table}[t]
\centering
\small
\setlength{\tabcolsep}{4pt}
\caption{Per-condition means for \textbf{"""+metric+r"""}. Bold indicates the best (including ties).}
\label{tab:per_condition_"""+metric+r"""}
\begin{tabular}{llrrrrrr}
\toprule
Dataset & Model & AIME & HuberAIME & RidgeAIME & HuberRidgeAIME & LIME & SHAP \\
\midrule
""")
    rows = [f"{d} & {m} & " + " & ".join(str(sty.loc[(d,m), c]) for c in method_order) + r" \\"
            for (d,m) in sty.index]
    tail = r"""
\bottomrule
\end{tabular}
\end{table}
"""
    (OUT/f'per_condition_{metric}.tex').write_text(hdr+"\n".join(rows)+tail, encoding='utf-8')

for met in ['stability','noise','decoy']: per_cond(met)

# ---------- Synthetic paired: HRA vs baselines (Wilcoxon one-sided, Holm) ----------
grid = csv_in('grid_raw.csv')
def paired(metric, base):
    hra = grid.loc[grid['method']=='HuberRidgeAIME', metric].reset_index(drop=True)
    oth = grid.loc[grid['method']==base, metric].reset_index(drop=True)
    n = min(len(hra), len(oth))
    x = hra.iloc[:n] - oth.iloc[:n]
    if np.allclose(x, 0.0):
        return n, hra.mean(), oth.mean(), 0.0, 0.0, 1.0, 0.0
    stat, p = stats.wilcoxon(x, zero_method='pratt', alternative='greater', mode='auto')
    Z = norm.isf(p); r = Z/np.sqrt(n); hl = np.median(x)
    return n, hra.mean(), oth.mean(), (hra.mean()-oth.mean()), hl, p, r

rows=[]
for met in ['stability','noise','decoy']:
    for base in ['AIME','HuberAIME','RidgeAIME']:
        rows.append((met, base, *paired(met, base)))

# ▼ ここから挿入：rows から DataFrame を作って Holm 補正し、列名を整理
paired_df = pd.DataFrame(
    rows,
    columns=['metric','baseline','n','mean_hra','mean_base','diff','hl','p','r']
)

# Holm 補正（各 metric 内で）
paired_df['p_holm'] = np.nan
for met in ['stability','noise','decoy']:
    idx = paired_df['metric'] == met
    pvals = paired_df.loc[idx, 'p'].to_numpy()
    order = np.argsort(pvals)
    adj = np.empty_like(pvals)
    m = len(pvals)
    for rank, j in enumerate(order, 1):
        adj[j] = min((m - rank + 1) * pvals[j], 1.0)
    paired_df.loc[idx, 'p_holm'] = adj

# 衝突回避＆可読性のために列名変更
paired_df = paired_df.rename(columns={'diff': 'mean_diff', 'r': 'r_effect'})
# ▲ ここまで

# 直前で作った paired_df に対して列名を変更
#paired_df = paired_df.rename(columns={'diff':'mean_diff', 'r':'r_effect'})

def f3(x): return f"{x:.3f}"
def fps(p): return r"$<10^{-4}$" if p<1e-4 else f"{p:.4f}"

lines = [r"""\begin{table*}[t]
\centering
\small
\setlength{\tabcolsep}{6pt}
\caption{Paired comparisons on the synthetic grid (one-sided Wilcoxon, alternative: HRA $>$ baseline).}
\label{tab:synthetic_paired}
\begin{tabular}{llr rrr r rrr}
\toprule
Metric & Baseline & $n$ & Mean(HRA) & Mean(Base) & Diff & HL & $p_{\mathrm{Wilc}}$ & $p_{\mathrm{Holm}}$ & $r$ \\
\midrule"""]

for _, row in paired_df.iterrows():
    lines.append(
        f"{row['metric']} & {row['baseline']} & {int(row['n'])} & "
        f"{f3(row['mean_hra'])} & {f3(row['mean_base'])} & "
        f"{f3(row['mean_diff'])} & {f3(row['hl'])} & "
        f"{fps(row['p'])} & {fps(row['p_holm'])} & {f3(row['r_effect'])} \\\\"
    )

lines.append(r"""\bottomrule
\end{tabular}
\end{table*}""")

(Path(OUT)/'synthetic_paired_clean.tex').write_text("\n".join(lines), encoding='utf-8')


#paired_df = pd.DataFrame(rows, columns=['metric','baseline','n','mean_hra','mean_base','diff','hl','p','r'])

# Holm per metric
#paired_df['p_holm'] = 0.0
#for met in ['stability','noise','decoy']:
#    idx = paired_df['metric']==met
#    p = paired_df.loc[idx,'p'].values
#    order = np.argsort(p); m=len(p); adj=np.empty_like(p)
#    for k,j in enumerate(order,1): adj[j]=min((m-k+1)*p[j], 1.0)
#    paired_df.loc[idx,'p_holm']=adj

#def f3(x): return f"{x:.3f}"
#def fps(p): return r"$<10^{-4}$" if p<1e-4 else f"{p:.4f}"

#1ines=[r"""\begin{table*}[t]
#\centering
#\small
#\setlength{\tabcolsep}{6pt}
#\caption{Paired comparisons on the synthetic grid (one-sided Wilcoxon, alternative: HRA $>$ baseline).}
#\label{tab:synthetic_paired}
#\begin{tabular}{llr rrr r rrr}
#\toprule
#Metric & Baseline & $n$ & Mean(HRA) & Mean(Base) & Diff & HL & $p_{\mathrm{Wilc}}$ & $p_{\mathrm{Holm}}$ & $r$ \\
#\midrule"""]
#for _,r in paired_df.iterrows():
#    lines.append(f"{r.metric} & {r.baseline} & {int(r.n)} & {f3(r.mean_hra)} & {f3(r.mean_base)} & {f3(r.diff)} & {f3(r.hl)} & {fps(r.p)} & {fps(r.p_holm)} & {f3(r.r)} \\\\")
#lines.append(r"""\bottomrule
#\end{tabular}
#\end{table*}""")
#(OUT/'synthetic_paired_clean.tex').write_text("\n".join(lines), encoding='utf-8')

# ---------- Panels (6 subplots: LIME, SHAP, AIME, HuberAIME, RidgeAIME, HuberRidgeAIME) ----------
def cwfi(dataset, model): 
    df = csv_in(f"cwfi_{dataset}__{model}.csv")
    df['method'] = pd.Categorical(df['method'], categories=method_order, ordered=True)
    return df

def make_panel(dataset, model, topk=15):
    df = cwfi(dataset, model)
    methods = [m for m in method_order if m in df['method'].unique()]
    ncols=2; nrows=int(math.ceil(len(methods)/ncols))
    fig = plt.figure(figsize=(12,14))
    fig.suptitle(f"Global Feature Importance Comparison (class-wise)\n({dataset} / {model})", fontsize=12)
    gs = fig.add_gridspec(nrows, ncols, hspace=0.25, wspace=0.25)
    for i,m in enumerate(methods):
        r=i//ncols; c=i%ncols; ax=fig.add_subplot(gs[r,c]); ax.set_title(f"Feature Importance by {m}")
        sub=df[df['method']==m].copy()
        agg=(sub.groupby('feature',as_index=False)['weight'].apply(lambda s:s.abs().mean())
                 .rename(columns={'weight':'abs_mean'}))
        top=agg.sort_values('abs_mean',ascending=False).head(topk)['feature'].tolist()
        plot=sub[sub['feature'].isin(top)]
        order=(plot.groupby('feature')['weight'].apply(lambda s:s.abs().mean())
                     .sort_values(ascending=False).index.tolist())
        classes=list(plot['cls'].unique()); y=np.arange(len(order)); width=0.4
        for j,cls in enumerate(classes):
            vals=[float(plot[(plot['feature']==f)&(plot['cls']==cls)]['weight'].iloc[0]) if not plot[(plot['feature']==f)&(plot['cls']==cls)].empty else 0.0 for f in order]
            ax.barh(y+(j-0.5)*width, vals, height=width, label=cls)
        ax.set_yticks(y); ax.set_yticklabels(order); ax.axvline(0, ls='--', lw=0.8); ax.set_xlabel("Normalized Importance (signed)")
        if i==0: ax.legend(title="Class", fontsize=8)
    out = OUT/f"panel_{dataset}_{model}.png"
    fig.savefig(out, dpi=200, bbox_inches='tight'); plt.close(fig)

for d,m in product(['breast_cancer','credit_approval','har'], ['lgbm','mlp','svm']):
    make_panel(d,m)


print("Done. Outputs in:", OUT)

Done. Outputs in: /Users/takafumi/Documents/Python/output/produce_out


In [31]:
def make_panel(dataset, model, topk=15):
    df = cwfi(dataset, model)
    methods = [m for m in method_order if m in df['method'].unique()]

    ncols = 2
    nrows = int(math.ceil(len(methods) / ncols))
    per_subplot_h = max(6.0, 0.48 * topk + 1.8)
    fig_w = 18.0
    fig_h = nrows * per_subplot_h
    fig = plt.figure(figsize=(fig_w, fig_h))
    fig.suptitle(f"Global Feature Importance Comparison (class-wise)\n({dataset} / {model})", fontsize=13)

    gs = fig.add_gridspec(
        nrows, ncols,
        left=0.46, right=0.985,
        wspace=0.42, hspace=0.42
    )

    for idx, m in enumerate(methods):
        r = idx // ncols
        c = idx % ncols
        ax = fig.add_subplot(gs[r, c])
        ax.set_title(f"Feature Importance by {m}", fontsize=12)

        sub = df[df['method'] == m].copy()

        # Top15 (abs-max)
        agg = (sub.groupby('feature', as_index=False)['weight']
                  .apply(lambda s: s.abs().max())
                  .rename(columns={'weight': 'abs_max'}))
        top_feats = agg.sort_values('abs_max', ascending=False).head(topk)['feature'].tolist()
        plot_df = sub[sub['feature'].isin(top_feats)].copy()

        # 並び（上から大→小）
        order = (plot_df.groupby('feature')['weight']
                          .apply(lambda s: s.abs().max())
                          .sort_values(ascending=False).index.tolist())

        classes = list(plot_df['cls'].unique())
        K = len(classes)

        y_center = np.arange(len(order))        # 各項目の中心 y
        width = 0.80 / max(K, 1)                # 総幅≈0.8 に収めるよう1本当たりの高さ

        # j 番目の棒は (j-(K-1)/2) だけ上下にずらす → グループの中心が y_center 上に来る
        for j, cls in enumerate(classes):
            offset = (j - (K - 1) / 2.0) * width
            vals = [
                float(
                    plot_df[(plot_df['feature'] == f) & (plot_df['cls'] == cls)]['weight'].iloc[0]
                ) if not plot_df[(plot_df['feature'] == f) & (plot_df['cls'] == cls)].empty else 0.0
                for f in order
            ]
            ax.barh(y_center + offset, vals, height=width, label=cls)

        # 軸と体裁
        ax.set_yticks(y_center)
        ax.set_yticklabels(order, fontsize=9)
        ax.invert_yaxis()  # 上からTop
        ax.tick_params(axis='x', labelsize=9)
        ax.margins(y=0.10)  # グループ上下に少し余白
        ax.axvline(0, linestyle='--', linewidth=0.8)
        ax.set_xlabel("Normalized Importance (signed)", fontsize=10)

        if idx == 0:
            ax.legend(title="Class", fontsize=9, title_fontsize=9,
                      frameon=False, loc='lower right')

    out = OUT / f"panel_{dataset}_{model}.png"
    fig.savefig(out, dpi=200, bbox_inches='tight')
    plt.close(fig)
    return out

In [32]:
for d, m in product(['breast_cancer','credit_approval','har'], ['lgbm','mlp','svm']):
    make_panel(d, m)

In [33]:
import sys; print(sys.version)

3.9.6 (default, Apr 30 2025, 02:07:17) 
[Clang 17.0.0 (clang-1700.0.13.5)]


In [34]:
# %% Supplemental: Rank & Top-K consistency (HRA vs baselines) and TeX table
from scipy.stats import spearmanr
import numpy as np
import pandas as pd
from pathlib import Path

import shap
from lime.lime_tabular import LimeTabularExplainer
print("shap version:", shap.__version__)
print("lime import OK")

# ---- 設定 ----
K_TOP = 10  # Top-K (CFG['top_k'] に合わせるなら: K_TOP = CFG.get('top_k', 10))
BASELINES = ['AIME','HuberAIME','RidgeAIME','LIME','SHAP']
TARGET = 'HuberRidgeAIME'  # 比較対象（HRA）

def load_cwfi(dataset, model):
    df = csv_in(f"cwfi_{dataset}__{model}.csv")
    df['method'] = pd.Categorical(df['method'], categories=method_order, ordered=True)
    return df

def agg_vectors(df_method):
    """
    クラスごとの重み 'weight' から特徴ベクトルを集約
    - abs_mean: 各特徴の |w| のクラス平均（順位付けに使用）
    - signed_mean: 各特徴の w のクラス平均（符号一致判定に使用）
    """
    g = df_method.groupby('feature')['weight']
    abs_mean = g.apply(lambda s: s.abs().mean())
    signed_mean = g.mean()
    return abs_mean, signed_mean

def topk_set(abs_mean, k):
    return set(abs_mean.sort_values(ascending=False).head(k).index.tolist())

rows = []
for dataset in ['breast_cancer','credit_approval','har']:
    for model in ['lgbm','mlp','svm']:
        df = load_cwfi(dataset, model)

        # HRA ベクトル
        hra_abs, hra_signed = agg_vectors(df[df['method']==TARGET])

        for base in BASELINES:
            if base not in df['method'].unique():
                continue

            base_abs, base_signed = agg_vectors(df[df['method']==base])

            # 共通の特徴に揃える（セーフティ）
            common_feats = hra_abs.index.intersection(base_abs.index)
            a = hra_abs.loc[common_feats]
            b = base_abs.loc[common_feats]

            # Spearman ρ（順位一致）
            rho, _ = spearmanr(a.values, b.values)

            # Top-K Jaccard
            A = topk_set(a, K_TOP)
            B = topk_set(b, K_TOP)
            jac = len(A & B) / max(1, len(A | B))

            # 符号一致率（平均符号の一致）
            hra_sign = np.sign(hra_signed.loc[common_feats].values)
            base_sign = np.sign(base_signed.loc[common_feats].values)
            sign_agree = (hra_sign == base_sign).mean()  # [-1,0,1] の 0 は“不一致”扱い

            rows.append([dataset, model, base, rho, jac, sign_agree])

cons = pd.DataFrame(rows, columns=['dataset','model','baseline','rho_spearman','jaccard_atK','sign_agree'])

# LaTeX 生成（1表に集約）
def f3(x): 
    try:
        return f"{float(x):.3f}"
    except Exception:
        return "--"

lines = [r"""\begin{table*}[t]
\centering
\small
\setlength{\tabcolsep}{6pt}
\caption{Consistency between \textbf{HRA} and baselines on global FI structures per dataset$\times$model. 
We report Spearman $\rho$ over feature importance ranks, Jaccard@K (K=""" + str(K_TOP) + r""") over top-$K$ features, and sign agreement of mean signed FI.}
\label{tab:hra_consistency}
\begin{tabular}{lllr rrr}
\toprule
Dataset & Model & Baseline & $\rho$ (rank) & Jaccard@K & Sign agree \\
\midrule"""]

for (d, m), sub in cons.groupby(['dataset','model'], sort=False):
    for _, r in sub.iterrows():
        lines.append(f"{d} & {m} & {r['baseline']} & {f3(r['rho_spearman'])} & {f3(r['jaccard_atK'])} & {f3(r['sign_agree'])} \\\\")
    lines.append(r"\midrule")

# 末尾の \midrule を \bottomrule に置換
if lines[-1] == r"\midrule":
    lines[-1] = r"\bottomrule"
else:
    lines.append(r"\bottomrule")
lines.append(r"\end{tabular}")
lines.append(r"\end{table*}")

tex_path = OUT / "hra_consistency.tex"
tex_path.write_text("\n".join(lines), encoding="utf-8")

print("Supplemental consistency table saved to:", tex_path)

shap version: 0.48.0
lime import OK
Supplemental consistency table saved to: /Users/takafumi/Documents/Python/output/produce_out/hra_consistency.tex


In [35]:
# %% Extended analysis: effect sizes + CIs, expanded consistency, local-stability proxy (optional)
# Assumes /mnt/data/result.zip exists (your earlier pipeline). Outputs go to /mnt/data/produce_out_ext/
import zipfile, io, math, os, textwrap, json, random
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import norm

import shap
from lime.lime_tabular import LimeTabularExplainer
print("shap version:", shap.__version__)
print("lime import OK")

# 1) まず方法名を確認
print("methods present:", sorted(grid['method'].unique()))
print(grid['method'].value_counts())

# 2) 名前のゆれ補正
name_map = {
    'KernelSHAP':'SHAP', 'kernelshap':'SHAP',
    'Lime':'LIME', 'lime':'LIME',
    'Huber Ridge AIME':'HuberRidgeAIME', 'HuberRidge':'HuberRidgeAIME',
}
grid['method'] = grid['method'].replace(name_map)

# 3) 再チェック
print("after map:", sorted(grid['method'].unique()))
print(grid['method'].value_counts())

# 4) LIME/SHAP が still 無いなら、AIME系のみ残してテーブル出力（LIME/SHAPは別図へ）
bases = ['AIME','HuberAIME','RidgeAIME']
if not set(['LIME','SHAP']).intersection(set(grid['method'].unique())):
    print("LIME/SHAP rows not found in grid_raw.csv -> 表は AIME-family のみにします（フロンティア図で補完）。")

ROOT = Path(OUT)
ZIP = "/Users/takafumi/Documents/Python/output/result.zip"
OUT = Path('/Users/takafumi/Documents/Python/output')
OUT.mkdir(exist_ok=True, parents=True)

z = zipfile.ZipFile(ZIP, 'r')

# ---------- 1) Wilcoxon (one-sided) + effect size r + bootstrap 95% CIs ----------
with z.open('grid_raw.csv') as f:
    grid = pd.read_csv(f)

def paired_wilcoxon_with_ci(metric, base, n_boot=2000, seed=42):
    rng = np.random.default_rng(seed)
    hra = grid.loc[grid['method']=='HuberRidgeAIME', metric].reset_index(drop=True).to_numpy()
    oth = grid.loc[grid['method']==base, metric].reset_index(drop=True).to_numpy()
    n = min(hra.size, oth.size)
    d = hra[:n] - oth[:n]
    if np.allclose(d, 0):
        w_stat, p = 0.0, 1.0; Z = 0.0
    else:
        w_stat, p = stats.wilcoxon(d, zero_method='pratt', alternative='greater', mode='auto')
        Z = norm.isf(p) if p>0 else np.inf
    r = (Z/np.sqrt(n)) if np.isfinite(Z) else 0.0
    mean_diff = float(np.mean(d)); hl = float(np.median(d))
    if n>1 and not np.allclose(d,0):
        idx = rng.integers(0, n, size=(n_boot, n))
        d_boot = d[idx]
        mean_boot = d_boot.mean(axis=1); hl_boot = np.median(d_boot, axis=1)
        ci_mean = (np.quantile(mean_boot, 0.025), np.quantile(mean_boot, 0.975))
        ci_hl   = (np.quantile(hl_boot, 0.025), np.quantile(hl_boot, 0.975))
    else:
        ci_mean = (0.0,0.0); ci_hl=(0.0,0.0)
    return dict(metric=metric, baseline=base, n=n,
                mean_hra=float(hra[:n].mean()), mean_base=float(oth[:n].mean()),
                mean_diff=mean_diff, hl=hl, p=p, r=r, w_stat=w_stat,
                ci_mean=ci_mean, ci_hl=ci_hl)

rows=[]
for metric in ['stability','noise','decoy']:
    for base in ['AIME','HuberAIME','RidgeAIME','HuberRidgeAIME','LIME','SHAP']:
        rows.append(paired_wilcoxon_with_ci(metric, base))
paired_df = pd.DataFrame(rows)

# Holm correction per metric
paired_df['p_holm']=0.0
for metric in ['stability','noise','decoy']:
    msk = paired_df['metric']==metric
    pvals = paired_df.loc[msk,'p'].to_numpy()
    order = np.argsort(pvals); adj=np.empty_like(pvals); m=len(pvals)
    for rank,j in enumerate(order,1):
        adj[j]=min((m-rank+1)*pvals[j],1.0)
    paired_df.loc[msk,'p_holm']=adj

def f3(x): return f"{x:.3f}"
def fps(p): return r"$<10^{-4}$" if p<1e-4 else f"{p:.4f}"

lines=[r"""\begin{table*}[t]
\centering
\small
\setlength{\tabcolsep}{6pt}
\caption{Paired comparisons on the synthetic grid (one-sided Wilcoxon, alternative: HRA $>$ baseline) with effect sizes and bootstrap CIs.}
\label{tab:synthetic_paired_ext}
\begin{tabular}{llr rrr r rr r}
\toprule
Metric & Baseline & $n$ & Mean(HRA) & Mean(Base) & Diff & HL & $p_{\mathrm{Wilc}}$ & $p_{\mathrm{Holm}}$ & $r$ \\
\midrule"""]
for _,r in paired_df.iterrows():
    ci_m=f"[{f3(r['ci_mean'][0])}, {f3(r['ci_mean'][1])}]"
    ci_h=f"[{f3(r['ci_hl'][0])}, {f3(r['ci_hl'][1])}]"
    lines.append(f"{r['metric']} & {r['baseline']} & {int(r['n'])} & {f3(r['mean_hra'])} & {f3(r['mean_base'])} & "
                 f"{f3(r['mean_diff'])}~({ci_m}) & {f3(r['hl'])}~({ci_h}) & {fps(r['p'])} & {fps(r['p_holm'])} & {f3(r['r'])} \\\\")
lines.append(r"""\bottomrule
\end{tabular}
\end{table*}""")
(OUT/'synthetic_paired_with_ci.tex').write_text("\n".join(lines), encoding='utf-8')

# ---------- 2) Consistency extended: add Kendall τ ----------
def read_csv_in_zip(name):
    with z.open(name) as f: return pd.read_csv(f)

datasets=['breast_cancer','credit_approval','har']
models=['lgbm','mlp','svm']
method_order=['AIME','HuberAIME','RidgeAIME','HuberRidgeAIME','LIME','SHAP']
from scipy.stats import kendalltau

def load_cwfi(dataset, model):
    df = read_csv_in_zip(f"cwfi_{dataset}__{model}.csv")
    df['method']=pd.Categorical(df['method'], categories=method_order, ordered=True)
    return df

def agg_abs_and_signed(df_method):
    g=df_method.groupby('feature')['weight']
    return g.apply(lambda s:s.abs().mean()), g.mean()

def topk(A,k): return set(A.sort_values(ascending=False).head(k).index.tolist())

rows=[]
K_TOP=10
for d in datasets:
    for m in models:
        df=load_cwfi(d,m)
        hra_abs, hra_signed = agg_abs_and_signed(df[df['method']=='HuberRidgeAIME'])
        for base in ['AIME','HuberAIME','RidgeAIME','LIME','SHAP']:
            if base not in df['method'].unique(): continue
            b_abs, b_signed = agg_abs_and_signed(df[df['method']==base])
            common = hra_abs.index.intersection(b_abs.index)
            s_rho,_ = stats.spearmanr(hra_abs.loc[common], b_abs.loc[common])
            k_tau,_ = kendalltau(hra_abs.loc[common], b_abs.loc[common])
            jacc = len(topk(hra_abs,K_TOP)&topk(b_abs,K_TOP))/max(1,len(topk(hra_abs,K_TOP)|topk(b_abs,K_TOP)))
            sign_agree = (np.sign(hra_signed.loc[common])==np.sign(b_signed.loc[common])).mean()
            rows.append([d,m,base,s_rho,k_tau,jacc,sign_agree])

cons2 = pd.DataFrame(rows, columns=['dataset','model','baseline','spearman_rho','kendall_tau','jaccard_atK','sign_agree'])

def f3n(x): 
    try: return f"{float(x):.3f}"
    except: return "--"

lines=[r"""\begin{table*}[t]
\centering
\small
\setlength{\tabcolsep}{6pt}
\caption{Consistency between \textbf{HRA} and baselines on global FI: Spearman $\rho$, Kendall $\tau$, Jaccard@K ($K=10$), and sign agreement.}
\label{tab:hra_consistency_ext}
\begin{tabular}{lllr rrrr}
\toprule
Dataset & Model & Baseline & $\rho$ & $\tau$ & Jaccard@K & Sign agree \\
\midrule"""]
for (_, _), sub in cons2.groupby(['dataset','model'], sort=False):
    for _,r in sub.iterrows():
        lines.append(f"{r['dataset']} & {r['model']} & {r['baseline']} & {f3n(r['spearman_rho'])} & "
                     f"{f3n(r['kendall_tau'])} & {f3n(r['jaccard_atK'])} & {f3n(r['sign_agree'])} \\\\")
    lines.append(r"\midrule")
if lines[-1]==r"\midrule": lines[-1]=r"\bottomrule"
else: lines.append(r"\bottomrule")
lines.append(r"\end{tabular}")
lines.append(r"\end{table*}")
(OUT/'hra_consistency_ext.tex').write_text("\n".join(lines), encoding='utf-8')

# ---------- 3) Optional local-stability function (use if LFI matrices are saved) ----------
def local_stability_metrics(lfi_base: np.ndarray, lfi_noise: np.ndarray, k=10):
    from scipy.stats import kendalltau
    assert lfi_base.shape == lfi_noise.shape
    n, d = lfi_base.shape
    j_scores, t_scores = [], []
    for i in range(n):
        b = pd.Series(lfi_base[i,:]); n_ = pd.Series(lfi_noise[i,:])
        top_b = set(b.abs().sort_values(ascending=False).head(k).index)
        top_n = set(n_.abs().sort_values(ascending=False).head(k).index)
        denom = len(top_b|top_n)
        j = len(top_b & top_n)/(denom if denom>0 else 1)
        t = kendalltau(b.values, n_.values)[0]
        j_scores.append(j); t_scores.append(t)
    j_scores=np.array(j_scores); t_scores=np.array(t_scores)
    return {
        'jaccard_atK_mean': float(np.nanmean(j_scores)),
        'jaccard_atK_std':  float(np.nanstd(j_scores)),
        'kendall_tau_mean': float(np.nanmean(t_scores)),
        'kendall_tau_std':  float(np.nanstd(t_scores)),
    }

(Path(OUT/'README_LOCAL_STABILITY.txt')
 .write_text("[Optional] Save LFI matrices per method (original & noise-perturbed) to compute instance-wise Top-K Jaccard & Kendall tau with local_stability_metrics(...).", encoding='utf-8'))

print("Saved:",
      OUT/'synthetic_paired_with_ci.tex',
      OUT/'hra_consistency_ext.tex',
      OUT/'README_LOCAL_STABILITY.txt')

shap version: 0.48.0
lime import OK
methods present: ['AIME', 'HuberAIME', 'HuberRidgeAIME', 'LINE', 'RidgeAIME', 'SHAP']
method
AIME              240
HuberAIME         240
RidgeAIME         240
HuberRidgeAIME    240
LINE              240
SHAP              240
Name: count, dtype: int64
after map: ['AIME', 'HuberAIME', 'HuberRidgeAIME', 'LINE', 'RidgeAIME', 'SHAP']
method
AIME              240
HuberAIME         240
RidgeAIME         240
HuberRidgeAIME    240
LINE              240
SHAP              240
Name: count, dtype: int64
Saved: /Users/takafumi/Documents/Python/output/synthetic_paired_with_ci.tex /Users/takafumi/Documents/Python/output/hra_consistency_ext.tex /Users/takafumi/Documents/Python/output/README_LOCAL_STABILITY.txt


In [36]:
print("methods present in grid_raw.csv:")
print(pd.Series(grid['method'].unique()))
print("\ncounts by method:")
print(grid['method'].value_counts(dropna=False))

methods present in grid_raw.csv:
0              AIME
1         HuberAIME
2         RidgeAIME
3    HuberRidgeAIME
4              LINE
5              SHAP
dtype: object

counts by method:
method
AIME              240
HuberAIME         240
RidgeAIME         240
HuberRidgeAIME    240
LINE              240
SHAP              240
Name: count, dtype: int64


In [37]:
# --- Paired tests with direction option, HL estimator, 95% CI, and Holm within-metric ---

import numpy as np, pandas as pd, math, time
from scipy import stats

def hodges_lehmann_and_ci(d, alpha=0.05):
    d = np.asarray(d, float)
    n = d.size
    walsh = np.fromiter((0.5*(d[i]+d[j]) for i in range(n) for j in range(i, n)), float, count=n*(n+1)//2)
    walsh.sort()
    hl = np.median(walsh)
    lo = np.quantile(walsh, alpha/2.0)
    hi = np.quantile(walsh, 1.0 - alpha/2.0)
    return float(hl), float(lo), float(hi)

def holm_stepdown(pvals):
    p = np.asarray(pvals, float)
    m = p.size
    order = np.argsort(p)
    adj = np.empty_like(p)
    for k, idx in enumerate(order):
        adj[idx] = (m - k) * p[idx]
    # step-down monotonicity
    adj_sorted = np.minimum.accumulate(adj[order][::-1])[::-1]
    out = np.empty_like(p)
    out[order] = np.minimum(1.0, adj_sorted)
    return out

def r_from_t(t_stat, n):
    if n is None or n<=1 or not np.isfinite(t_stat): return np.nan
    return float(t_stat / math.sqrt(t_stat*t_stat + (n-1)))

def r_from_wilcoxon_p(p_one_sided, n):
    if not np.isfinite(p_one_sided) or n is None or n<=0: return np.nan
    z = stats.norm.isf(p_one_sided)  # one-sided
    return float(z / math.sqrt(n))

def paired_grid_stats(grid_csv, hra_name='HuberRidgeAIME',
                      baselines=('AIME','HuberAIME','RidgeAIME'),
                      metrics=('decoy','noise','stability'),
                      direction='greater'):  # 'greater' or 'two-sided'
    df = pd.read_csv(grid_csv)
    out_rows = []
    for met in metrics:
        rows_this_metric, p_t_list, p_w_list = [], [], []
        for base in baselines:
            sub = df[df['method'].isin([hra_name, base])]
            pv  = sub.pivot_table(index=['rho','pi','rep'], columns='method', values=met, aggfunc='mean').dropna()
            if hra_name not in pv or base not in pv: 
                continue
            hra, bse = pv[hra_name].to_numpy(), pv[base].to_numpy()
            diff = hra - bse; n = diff.size
            # t-test
            try:
                if direction == 'greater':
                    t_stat, p_t = stats.ttest_rel(hra, bse, alternative='greater')
                else:
                    t_stat, p_t = stats.ttest_rel(hra, bse)  # two-sided
                t_stat, p_t = float(t_stat), float(p_t)
            except TypeError:  # SciPy < 1.9 fallback
                t_stat, p2 = stats.ttest_rel(hra, bse)
                p_t = p2/2.0 if (direction=='greater' and t_stat>0) else (1.0 - p2/2.0 if direction=='greater' else p2)
                t_stat, p_t = float(t_stat), float(p_t)
            # Wilcoxon
            try:
                if direction == 'greater':
                    w_stat, p_w = stats.wilcoxon(hra, bse, alternative='greater', zero_method='pratt')
                else:
                    w_stat, p_w = stats.wilcoxon(hra, bse, alternative='two-sided', zero_method='pratt')
                p_w = float(p_w)
            except Exception:
                w_stat, p_w = (np.nan, np.nan)
            # HL & CI
            hl, lo, hi = hodges_lehmann_and_ci(diff, alpha=0.05)
            rows_this_metric.append(dict(
                metric=met, baseline=base, n=n,
                mean_HRA=float(hra.mean()), mean_base=float(bse.mean()),
                mean_diff=float(diff.mean()), hl=hl, ci_lo=lo, ci_hi=hi,
                t_stat=t_stat, p_t=p_t, r_t=r_from_t(t_stat, n),
                w_stat=(float(w_stat) if np.isfinite(w_stat) else np.nan),
                p_wilcoxon=p_w, r_w=r_from_wilcoxon_p(p_w, n)
            ))
            p_t_list.append(p_t); p_w_list.append(p_w if np.isfinite(p_w) else 1.0)
        # Holm within metric
        if rows_this_metric:
            adj_t = holm_stepdown(np.array(p_t_list))
            adj_w = holm_stepdown(np.array(p_w_list))
            for row, pt_adj, pw_adj in zip(rows_this_metric, adj_t, adj_w):
                row['p_t_holm'] = float(pt_adj)
                row['p_wilcoxon_holm'] = float(pw_adj)
                out_rows.append(row)
    return pd.DataFrame(out_rows)

In [38]:
# --- Cost–Performance Frontier for LIME and SHAP on one mid-sized condition ---

import time, psutil, os
from matplotlib import pyplot as plt

def measure_frontier(dataset_loader, model_builder, class_names,
                     lime_num_samples_list=(1000, 3000, 7000),
                     shap_budgets=((64, 50, 500), (128, 100, 1000), (256, 150, 2000)),
                     k_top=10, noise_frac=0.10, rs=42, out_png=None):
    """
    dataset_loader(): returns (X, y)
    model_builder(): returns fitted pipeline with predict_proba
    class_names: list[str]
    shap_budgets: list of tuples (m_sample, bg_n, nsamples)
    """
    X, y = dataset_loader()
    pipe  = model_builder()
    proc  = psutil.Process(os.getpid())
    results = []

    # helper metrics
    def _topk(V, k): 
        g = np.abs(V).sum(axis=1); k = min(k, g.size)
        return set(np.argpartition(-g, k-1)[:k].tolist())
    def _stability(expl, X, B=5):
        base = _topk(expl(X), k_top); n = X.shape[0]
        if not base: return 0.0
        sc = []
        rng = np.random.RandomState(rs)
        for _ in range(B):
            idx = rng.choice(n, size=n, replace=True)
            sc.append(len(base & _topk(expl(X[idx]), k_top)) / len(base | _topk(expl(X[idx]), k_top)))
        return float(np.mean(sc))
    def _noise_robust(expl, X, trials=5):
        base = _topk(expl(X), k_top); n = X.shape[0]
        if not base: return 0.0
        sc = []
        rng = np.random.RandomState(rs)
        std = X.std(axis=0, keepdims=True); std = np.where(std < 1e-12, 1.0, std)
        for _ in range(trials):
            Xn = X + rng.normal(0.0, noise_frac*std, size=X.shape)
            sc.append(len(base & _topk(expl(Xn), k_top)) / len(base | _topk(expl(Xn), k_top)))
        return float(np.mean(sc))

    # LIME
    for ns in lime_num_samples_list:
        t0 = time.perf_counter()
        mem0 = proc.memory_info().rss
        expl = lambda XX: cwfi_lime_signed(pipe, XX, class_names, sample_n=min(256, XX.shape[0]),
                                           num_samples=ns, kernel_width=0.75*np.sqrt(XX.shape[1]),
                                           random_state=rs)
        stab = _stability(expl, X, B=5)
        nrob = _noise_robust(expl, X, trials=5)
        dt = time.perf_counter()-t0
        mem = proc.memory_info().rss - mem0
        results.append(dict(method='LIME', budget=ns, stability=stab, noise=nrob, time_s=dt, mem_B=max(0, mem)))

    # SHAP (Kernel for non-tree, Tree for LGBM)
    C = len(class_names)
    for m_sample, bg_n, nsamples in shap_budgets:
        t0 = time.perf_counter()
        mem0 = proc.memory_info().rss
        expl = lambda XX: cwfi_shap_signed(pipe, XX, C, m_sample=min(m_sample, XX.shape[0]),
                                           bg_n=min(bg_n, XX.shape[0]), kernel_nsamples=nsamples,
                                           random_state=rs)
        stab = _stability(expl, X, B=5)
        nrob = _noise_robust(expl, X, trials=5)
        dt = time.perf_counter()-t0
        mem = proc.memory_info().rss - mem0
        results.append(dict(method='SHAP', budget=f"{m_sample}/{bg_n}/{nsamples}",
                            stability=stab, noise=nrob, time_s=dt, mem_B=max(0, mem)))

    df = pd.DataFrame(results)
    if out_png:
        fig = plt.figure(figsize=(6,4))
        for m in df['method'].unique():
            sub = df[df['method']==m]
            plt.plot(sub['time_s'], sub['stability'], marker='o', label=f"{m}-stability")
            plt.plot(sub['time_s'], sub['noise'], marker='x', label=f"{m}-noise")
        plt.xlabel("Wall-clock time (s)"); plt.ylabel("Score (Jaccard)")
        plt.legend(); plt.tight_layout(); plt.savefig(out_png, dpi=200); plt.close(fig)
    return df

In [39]:
# --- Decoy modes and graded metrics ---

def add_decoy(X, mode='correlated', rs=42):
    rng = np.random.RandomState(rs)
    X = np.asarray(X, float)
    n, d = X.shape
    if mode == 'permute':
        j = rng.randint(0, d); decoy = rng.permutation(X[:, j])
    elif mode == 'correlated':
        j1, j2 = rng.randint(0, d, size=2)
        noise = rng.normal(0.0, 0.1*np.std(X[:, j1])+1e-12, size=n)
        decoy = 0.75*X[:, j1] + 0.25*X[:, j2] + noise  # strong correlation
    elif mode == 'interaction':
        j1, j2 = rng.randint(0, d, size=2)
        decoy = (X[:, j1]*X[:, j2]) + rng.normal(0.0, 0.05, size=n)
    else:
        raise ValueError(mode)
    return np.column_stack([X, decoy])

def decoy_metrics(expl_fn, X, k=10, modes=('permute','correlated','interaction'), rs=42, trials=20):
    rng = np.random.RandomState(rs)
    X = np.asarray(X, float); d = X.shape[1]
    def _topk(V, k): 
        g = np.abs(V).sum(axis=1); k = min(k, g.size)
        return np.argpartition(-g, k-1)[:k]
    rows = []
    for mode in modes:
        infil = 0; mean_rank = []
        for _ in range(trials):
            Xa = add_decoy(X, mode=mode, rs=rng.randint(0, 10**9))
            V  = expl_fn(Xa)                  # explainer must tolerate extra column
            g  = np.abs(V).sum(axis=1)
            rank = int(np.argsort(-g).tolist().index(d))  # appended decoy's rank (0=best)
            mean_rank.append(rank)
            if d in _topk(V, k=k): infil += 1
        rows.append(dict(mode=mode,
                         infiltration_rate=infil/trials,
                         mean_decoy_rank=float(np.mean(mean_rank))))
    return pd.DataFrame(rows)

In [40]:
# Paths & discovery: find where stress_*.csv / cwfi_*.csv live, or create a fresh OUTDIR.
from pathlib import Path
import glob, os

def find_results_dir() -> Path:
    candidates = [
        "./HuberRidgeAIME_results",
        "./output/",
        "./result/",
        "./",
    ]
    hits = []
    for base in candidates:
        base = Path(base)
        if not base.exists():
            continue
        # look for stress_*.csv or cwfi_*.csv
        gl1 = list(base.glob("stress_*.csv")) + list(base.glob("**/stress_*.csv"))
        gl2 = list(base.glob("cwfi_*.csv")) + list(base.glob("**/cwfi_*.csv"))
        if gl1 or gl2:
            # choose the deepest directory that contains most files
            # (simple heuristic: use the parent directory of the most matches)
            if gl1:
                hits.append(Path(gl1[0]).parent)
            elif gl2:
                hits.append(Path(gl2[0]).parent)
    if hits:
        # prefer the path that looks like .../HuberRidgeAIME_results
        hits = sorted(hits, key=lambda p: (str(p).count("HuberRidgeAIME_results")==0, -len(list(p.glob('**/*')))))
        return hits[0]
    # fallback: create new
    fresh = Path("/content/HuberRidgeAIME_results")
    fresh.mkdir(parents=True, exist_ok=True)
    return fresh

OUTDIR = find_results_dir()
OUTDIR.mkdir(parents=True, exist_ok=True)
print("[OUTDIR]", OUTDIR.resolve())

# Quick listing
import subprocess, shlex
try:
    subprocess.run(shlex.split(f'ls -lah "{OUTDIR}"'), check=False)
except Exception:
    pass

[OUTDIR] /Users/takafumi/Documents/Python/HuberRidgeAIME/output/result
total 17336
drwx------@ 34 takafumi  staff   1.1K Sep 19 03:53 .
drwxr-xr-x@ 10 takafumi  staff   320B Sep 19 03:53 ..
-rw-r--r--@  1 takafumi  staff   8.0K Sep 21 16:40 .DS_Store
-rw-r--r--   1 takafumi  staff    41K Sep 12 18:02 ablation_decoy_HRA_minus_HuberAIME.png
-rw-r--r--   1 takafumi  staff    76K Sep 12 13:19 cwfi_breast_cancer__lgbm.csv
-rw-r--r--   1 takafumi  staff    50K Sep 12 14:00 cwfi_breast_cancer__mlp.csv
-rw-r--r--   1 takafumi  staff    75K Sep 12 13:26 cwfi_breast_cancer__svm.csv
-rw-r--r--   1 takafumi  staff    59K Sep 12 14:03 cwfi_credit_approval__lgbm.csv
-rw-r--r--   1 takafumi  staff    58K Sep 12 16:43 cwfi_credit_approval__mlp.csv
-rw-r--r--   1 takafumi  staff    58K Sep 12 14:20 cwfi_credit_approval__svm.csv
-rw-r--r--   1 takafumi  staff   2.4M Sep 12 16:52 cwfi_har__lgbm.csv
-rw-r--r--   1 takafumi  staff   2.3M Sep 12 17:58 cwfi_har__mlp.csv
-rw-r--r--   1 takafumi  staff   2.3M 

In [41]:
# Harvest: build LaTeX tables and zip all artifacts from OUTDIR.
import pandas as pd, numpy as np, math, zipfile, os
from pathlib import Path
from scipy import stats

def build_overall_mean_std(OUTDIR: Path) -> pd.DataFrame:
    frames=[]
    for p in OUTDIR.glob("stress_*.csv"):
        try: frames.append(pd.read_csv(p))
        except: pass
    if not frames:
        raise FileNotFoundError("No stress_*.csv found under: " + str(OUTDIR))
    allst = pd.concat(frames, axis=0, ignore_index=True)
    per_cond = (allst.groupby(["dataset","model","method"], as_index=False)[["stability","noise","decoy"]].mean())
    overall = (per_cond.groupby("method")[["stability","noise","decoy"]].agg(["mean","std"]).reset_index())
    # flatten MultiIndex -> method, stability_mean, stability_std, ...
    overall.columns = ["method"] + [f"{a}_{b}" for (a,b) in overall.columns.tolist()[1:]]
    # reorder methods
    order = ["AIME","HuberAIME","RidgeAIME","HuberRidgeAIME","LIME","SHAP"]
    overall["order"] = overall["method"].apply(lambda m: order.index(m) if m in order else 999)
    overall = overall.sort_values("order").drop(columns="order")
    return overall

def fmt_pm(mu, sd, nd=3): return f"{mu:.{nd}f} $\\pm$ {sd:.{nd}f}"

def tex_overall_mean_std(overall: pd.DataFrame) -> str:
    lines = [
        "\\begin{table}[t]",
        "\\centering",
        "\\caption{Overall averages over 9 conditions (3 datasets $\\times$ 3 models). Mean$\\pm$std.}",
        "\\label{tab:overall_mean_std}",
        "\\small",
        "\\setlength{\\tabcolsep}{6pt}",
        "\\begin{tabular}{lccc}",
        "\\toprule",
        "\\textbf{Method} & \\textbf{Stability} & \\textbf{Noise Robust.} & \\textbf{Decoy Resist.} \\\\",
        "\\midrule",
    ]
    for _, r in overall.iterrows():
        s = fmt_pm(r["stability_mean"], r["stability_std"])
        n = fmt_pm(r["noise_mean"], r["noise_std"])
        d = fmt_pm(r["decoy_mean"], r["decoy_std"])
        lines.append(f"{r['method']} & {s} & {n} & {d} \\\\")
    lines += ["\\bottomrule", "\\end{tabular}", "\\end{table}"]
    return "\n".join(lines)

def zip_all(OUTDIR: Path) -> Path:
    zpath = OUTDIR.parent / "result.zip"
    with zipfile.ZipFile(zpath, "w", zipfile.ZIP_DEFLATED) as zf:
        for root, _, files in os.walk(OUTDIR):
            for f in files:
                fp = os.path.join(root, f)
                arc = os.path.relpath(fp, start=OUTDIR)
                zf.write(fp, arcname=arc)
    return zpath

OUTDIR = Path(str(OUTDIR))  # from cell 2
tables_dir = OUTDIR / "tables"; tables_dir.mkdir(exist_ok=True)

overall = build_overall_mean_std(OUTDIR)
tex1 = tex_overall_mean_std(overall)
with open(tables_dir / "overall_mean_std.tex", "w") as f:
    f.write(tex1)

zpath = zip_all(OUTDIR)

print("[saved]", tables_dir / "overall_mean_std.tex")
print("[zip]", zpath)
# Quick listing
import subprocess, shlex
subprocess.run(shlex.split(f'ls -lah "{tables_dir}"'), check=False)

[saved] output/result/tables/overall_mean_std.tex
[zip] output/result.zip
total 376
drwxr-xr-x  22 takafumi  staff   704B Sep 19 03:53 .
drwx------@ 34 takafumi  staff   1.1K Sep 19 03:53 ..
-rw-r--r--@  1 takafumi  staff   178B Sep 19 15:16 avg_ranks_decoy.csv
-rw-r--r--@  1 takafumi  staff   354B Sep 19 15:16 avg_ranks_decoy.tex
-rw-r--r--@  1 takafumi  staff   162B Sep 19 15:16 avg_ranks_noise.csv
-rw-r--r--@  1 takafumi  staff   354B Sep 19 15:16 avg_ranks_noise.tex
-rw-r--r--@  1 takafumi  staff   163B Sep 19 15:16 avg_ranks_stability.csv
-rw-r--r--@  1 takafumi  staff   362B Sep 19 15:16 avg_ranks_stability.tex
-rw-r--r--@  1 takafumi  staff    26K Sep 19 15:16 cd_decoy.png
-rw-r--r--@  1 takafumi  staff    26K Sep 19 15:16 cd_noise.png
-rw-r--r--@  1 takafumi  staff    27K Sep 19 15:16 cd_stability.png
-rw-r--r--@  1 takafumi  staff    38K Sep 19 15:16 cd_triptych.png
-rw-r--r--@  1 takafumi  staff   324B Sep 19 15:16 friedman_nemenyi_decoy.tex
-rw-r--r--@  1 takafumi  staff   3

CompletedProcess(args=['ls', '-lah', 'output/result/tables'], returncode=0)

--@  1 takafumi  staff   331B Sep 19 15:16 friedman_nemenyi_stability.tex
-rw-r--r--@  1 takafumi  staff   179B Sep 19 15:16 nemenyi_nonsig_decoy.csv
-rw-r--r--@  1 takafumi  staff   179B Sep 19 15:16 nemenyi_nonsig_noise.csv
-rw-r--r--@  1 takafumi  staff   179B Sep 19 15:16 nemenyi_nonsig_stability.csv
-rw-r--r--@  1 takafumi  staff   778B Sep 21 21:47 overall_mean_std.tex
-rw-r--r--   1 takafumi  staff   773B Sep 12 18:02 overall_mean_std_preview.csv
-rw-r--r--@  1 takafumi  staff   1.8K Sep 12 18:02 synthetic_stats.tex
-rw-r--r--   1 takafumi  staff   2.0K Sep 12 18:02 synthetic_stats_raw.csv


In [42]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.calibration import CalibratedClassifierCV
try:
    from lightgbm import LGBMClassifier
    LGBM_OK = True
except Exception:
    LGBM_OK = False

def fit_learners(X, y, learner='svm', seed=42):
    Xtr, Xva, ytr, yva = train_test_split(X, y, test_size=0.2, random_state=seed, stratify=y)
    if learner=='lgbm' and LGBM_OK:
        clf = LGBMClassifier(n_estimators=300, learning_rate=0.05, max_depth=10,
                             subsample=0.9, colsample_bytree=0.9, random_state=seed)
        clf.fit(Xtr, ytr, eval_set=[(Xva,yva)], eval_metric='logloss')
    elif learner=='svm':
        base = SVC(C=10, kernel='rbf', probability=True, random_state=seed)
        clf = CalibratedClassifierCV(base, method='sigmoid', cv=3)
        clf.fit(Xtr, ytr)
    elif learner=='mlp':
        clf = MLPClassifier(hidden_layer_sizes=(100,50), activation='relu',
                            learning_rate_init=1e-3, max_iter=200,
                            random_state=seed, early_stopping=True, n_iter_no_change=10)
        clf.fit(Xtr, ytr)
    else:
        raise ValueError("Unknown learner")
    # 全データに対する予測確率を返す
    return clf, clf.predict_proba(X)

In [43]:
try:
    import shap
    from lime.lime_tabular import LimeTabularExplainer
    SHAP_OK=True; LIME_OK=True
except Exception:
    SHAP_OK=False; LIME_OK=False

def evaluate_surrogates_frontier(X, y, learner='svm', budgets=(100,500,2000,8000), k_top=10, n_points=30, seed=42):
    rows=[]; rng=np.random.default_rng(seed); clf,_=fit_learners(X,y,learner,seed)
    idx=rng.choice(len(X), size=min(n_points,len(X)), replace=False)
    def topk_from_contribs(contribs):
        V=np.mean(np.vstack(contribs),axis=0); return set(np.argsort(-V)[:k_top].tolist()), V
    # LIME
    if LIME_OK:
        expl=LimeTabularExplainer(X,feature_names=[f'f{i}' for i in range(X.shape[1])],class_names=[str(c) for c in np.unique(y)],discretize_continuous=False,random_state=seed)
        for b in budgets:
            t0=time.time(); contribs=[]
            for i in idx:
                exp=expl.explain_instance(X[i], clf.predict_proba, num_features=X.shape[1], num_samples=b)
                v=np.zeros(X.shape[1])
                for fname,w in exp.as_list(label=1 if len(np.unique(y))>1 else 0):
                    if isinstance(fname,str) and fname.startswith('f'):
                        try:
                            j=int(fname[1:]); v[j]=abs(w)
                        except Exception:
                            pass
                contribs.append(v)
            base,V=topk_from_contribs(contribs); scores=[]; rng2=np.random.default_rng(seed)
            for _ in range(30):
                Vp=V+rng2.normal(0,0.01,size=V.shape); comp=set(np.argsort(-Vp)[:k_top].tolist()); scores.append(len(base&comp)/len(base|comp))
            rows.append({'method':'LIME','budget':int(b),'runtime_sec':time.time()-t0,'stability':np.nan,'noise':float(np.mean(scores))})
    # SHAP
    if SHAP_OK:
        for b in budgets:
            t0=time.time(); bg_idx=rng.choice(len(X), size=min(b,len(X)), replace=False); background=X[bg_idx]
            expl=shap.KernelExplainer(clf.predict_proba, background); contribs=[]
            for i in idx:
                sv=expl.shap_values(X[i:i+1], nsamples=b); v=np.abs(sv[1] if isinstance(sv,list) else sv).ravel(); contribs.append(v)
            base,V=topk_from_contribs(contribs); scores=[]; rng2=np.random.default_rng(seed)
            for _ in range(30):
                Vp=V+rng2.normal(0,0.01,size=V.shape); comp=set(np.argsort(-Vp)[:k_top].tolist()); scores.append(len(base&comp)/len(base|comp))
            rows.append({'method':'KernelSHAP','budget':int(b),'runtime_sec':time.time()-t0,'stability':np.nan,'noise':float(np.mean(scores))})
    return pd.DataFrame(rows)

# Example (commented):
X, y, _, _ = load_breast_cancer_dataset()
frontier = evaluate_surrogates_frontier(X, y, learner='svm')
frontier.to_csv(OUT/'frontier_wdbc_svm.csv', index=False)
frontier.head()

100%|██████████| 1/1 [00:00<00:00, 12.92it/s]
Using 500 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.
100%|██████████| 1/1 [00:01<00:00,  1.80s/it]
Using 569 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.
100%|██████████| 1/1 [00:08<00:00,  8.07s/it]
Using 569 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.
100%|██████████| 1/1 [00:31<00:00, 31.93s/it]


,method,budget,runtime_sec,stability,noise
0,LIME,100,0.072134,NaN,0.381807
1,LIME,500,0.175454,NaN,0.315593
2,LIME,2000,0.525791,NaN,0.269902
3,LIME,8000,2.036967,NaN,0.318414
4,KernelSHAP,100,2.402947,NaN,0.347985


In [45]:
df = measure_frontier(
    dataset_loader=..., model_builder=...,
    lime_num_samples_list=(1000, 3000, 7000),
    shap_budgets=((64,50,500),(128,100,1000),(256,150,2000)),
    out_png=OUT/'frontier_wdbc_svm.png'
)

TypeError: measure_frontier() missing 1 required positional argument: 'class_names'

In [ ]:
# %% [Friedman–Nemenyi: CD diagrams + LaTeX tables — robust/no-posthocs]
# Robust pipeline to:
#  - read stress_*.csv,
#  - coerce metrics to float,
#  - aggregate duplicates (task×method),
#  - restrict to tasks where ALL methods are present,
#  - run Friedman test (SciPy),
#  - compute Nemenyi non-significance via the CD (critical difference) threshold,
#  - draw CD diagrams and write LaTeX/CSV outputs under <RESULT_DIR>/tables/.

import os, pathlib, warnings, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

# ---------- locate result dir ----------
def _autodetect_result_dir():
    # Prefer OUTDIR if defined and contains stress_*.csv
    try:
        rd = pathlib.Path(OUTDIR)
        if rd.exists() and list(rd.glob("stress_*.csv")):
            return rd
    except NameError:
        pass
    # Fallbacks
    cands = [
        pathlib.Path("../output/HuberRidgeAIME_results"),
        pathlib.Path("../output/result"),
        pathlib.Path.cwd(),
        pathlib.Path("../output"),
    ]
    for d in cands:
        if d.exists() and list(d.glob("stress_*.csv")):
            return d
        for dd in d.glob("*"):
            if dd.is_dir() and list(dd.glob("stress_*.csv")):
                return dd
    raise FileNotFoundError("Could not find stress_*.csv. Set RESULT_DIR manually.")

RESULT_DIR = _autodetect_result_dir()
TABLE_DIR = RESULT_DIR / "tables"
TABLE_DIR.mkdir(parents=True, exist_ok=True)
print("[result dir]", RESULT_DIR)

# ---------- configuration ----------
METHODS = ["AIME", "HuberAIME", "RidgeAIME", "HuberRidgeAIME", "LIME", "SHAP"]
METRICS = ["stability", "noise", "decoy"]  # higher is better

def _normalize_method(m):
    m = str(m).strip()
    if m.upper() == "LINE":  # typo -> LIME
        return "LIME"
    return m

def _to_float(s):
    # strict numeric coercion; anything non-numeric becomes NaN
    return pd.to_numeric(s, errors="coerce")

# ---------- load & sanitize ----------
def load_scores(result_dir: pathlib.Path) -> pd.DataFrame:
    rows = []
    for p in result_dir.glob("stress_*.csv"):
        try:
            df = pd.read_csv(p)
        except Exception:
            continue
        # keep only expected columns
        keep = ["dataset","model","method","stability","noise","decoy"]
        cols = [c for c in keep if c in df.columns]
        df = df[cols].copy()
        # normalize method names
        if "method" in df.columns:
            df["method"] = df["method"].map(_normalize_method)
        # coerce metrics to float
        for col in METRICS:
            if col in df.columns:
                df[col] = _to_float(df[col])
        # drop rows with missing essentials
        df = df.dropna(subset=["dataset","model","method"])
        rows.append(df)
    if not rows:
        raise FileNotFoundError("No stress_*.csv could be read.")

    allscores = pd.concat(rows, axis=0, ignore_index=True)

    # build task label (dataset__model)
    allscores["task"] = allscores["dataset"].astype(str) + "__" + allscores["model"].astype(str)

    # keep only target METHODS
    allscores = allscores[allscores["method"].isin(METHODS)].copy()

    # aggregate duplicates by mean (task × method)
    agg = (allscores
           .groupby(["task","dataset","model","method"], as_index=False)[METRICS]
           .mean())
    return agg

scores = load_scores(RESULT_DIR)
print("[loaded rows]", len(scores))

# ---------- restrict to tasks that contain ALL METHODS ----------
def tasks_with_all_methods(df: pd.DataFrame, methods: list[str]) -> list[str]:
    g = df.groupby("task")["method"].agg(lambda s: set(s.unique()))
    return [t for t, mset in g.items() if set(methods).issubset(mset)]

valid_tasks = tasks_with_all_methods(scores, METHODS)
if not valid_tasks:
    raise RuntimeError("No task has all METHODS. Compute missing runs or relax METHODS.")
scores = scores[scores["task"].isin(valid_tasks)].copy()
N = len(set(scores["task"]))
k = len(METHODS)
print(f"[design] N tasks = {N}, k methods = {k}")

# ---------- Friedman test (SciPy) ----------
try:
    from scipy import stats
    from scipy.stats import studentized_range
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "scipy"])
    from scipy import stats
    from scipy.stats import studentized_range

def metric_matrix(df_long: pd.DataFrame, metric: str) -> pd.DataFrame:
    """Return an N×k matrix indexed by task with columns METHODS, numeric-only."""
    mat = (df_long.pivot_table(index="task", columns="method", values=metric, aggfunc="mean"))
    mat = mat.reindex(columns=METHODS)            # ensure column order
    mat = mat.apply(pd.to_numeric, errors="coerce")
    mat = mat.dropna(axis=0, how="any")          # drop tasks missing any method
    return mat

def average_ranks_for_metric(df_long: pd.DataFrame, metric: str) -> pd.Series:
    mat = metric_matrix(df_long, metric)         # (N,k)
    ranks = mat.rank(axis=1, ascending=False, method="average")  # rank 1 = best
    avg = ranks.mean(axis=0)
    return avg.reindex(METHODS)

def friedman_statistic(df_long: pd.DataFrame, metric: str):
    mat = metric_matrix(df_long, metric)
    groups = [mat[m].values for m in METHODS]    # lists of length N
    stat, p = stats.friedmanchisquare(*groups)
    return float(stat), float(p), mat.shape[0]   # also return effective N after drops

def compute_CD(alpha: float, k: int, N: int) -> float:
    q_alpha = studentized_range.isf(alpha, k, np.inf)
    return float(q_alpha * math.sqrt(k*(k+1) / (6.0*N)))

# ---------- CD-based non-significance matrix ----------
def nemenyi_nonsig_mask(avg_ranks: pd.Series, N: int, alpha: float=0.05) -> pd.DataFrame:
    """Return a k×k boolean matrix M where M[i,j]=True means NOT significantly different (|ri-rj| <= CD)."""
    CD = compute_CD(alpha, len(avg_ranks), N)
    idx = avg_ranks.index
    M = pd.DataFrame(False, index=idx, columns=idx)
    for a in idx:
        for b in idx:
            if a == b:
                M.loc[a, b] = True
            else:
                M.loc[a, b] = (abs(avg_ranks[a] - avg_ranks[b]) <= CD)
    return M

# ---------- CD diagram ----------
def plot_cd_diagram(avg_ranks: pd.Series,
                    nonsig: pd.DataFrame,
                    N: int, alpha: float,
                    out_png: pathlib.Path,
                    title: str):
    """Draw a CD diagram using the CD-based non-significance mask."""
    k = len(avg_ranks)
    CD = compute_CD(alpha, k, N)
    order = np.argsort(avg_ranks.values)
    methods_sorted = avg_ranks.index.values[order]
    ranks_sorted   = avg_ranks.values[order]

    fig, ax = plt.subplots(figsize=(8, 2.2), dpi=200)
    ax.set_title(title, fontsize=10)
    ax.set_xlim(1 - 0.2, k + 0.2)
    ax.set_ylim(0, 1)
    ax.get_yaxis().set_visible(False)

    # axis 1..k
    ax.hlines(0.5, 1, k, lw=1.0, color="black")
    for r in range(1, k+1):
        ax.vlines(r, 0.5-0.03, 0.5+0.03, color="black", lw=1.0)
        ax.text(r, 0.5+0.06, f"{r}", ha="center", va="bottom", fontsize=8)

    # CD bar
    ax.hlines(0.9, 1, 1+CD, lw=2.0, color="black")
    ax.vlines(1, 0.88, 0.92, lw=1.5, color="black")
    ax.vlines(1+CD, 0.88, 0.92, lw=1.5, color="black")
    ax.text(1 + CD/2.0, 0.93, f"CD={CD:.2f}", ha="center", va="bottom", fontsize=8)

    # points and labels
    y0 = 0.4
    for i, (m, r) in enumerate(zip(methods_sorted, ranks_sorted)):
        ax.plot([r], [y0], "o", ms=4, color="black")
        dy = 0.10 if (i % 2 == 0) else -0.12
        ax.text(r, y0 + dy, m, ha="center", va="center", fontsize=8)

    # non-significant cliques (greedy, adjacent spans)
    def all_nonsig(i, j):
        idx = methods_sorted[i:j+1]
        for a in range(len(idx)):
            for b in range(a+1, len(idx)):
                if not nonsig.loc[idx[a], idx[b]]:
                    return False
        return True

    bar_levels = []
    for span in range(1, k):
        for i in range(0, k-span):
            j = i + span
            if all_nonsig(i, j):
                level = 0.72
                while any(abs(level - L) < 0.06 and not (j < i2 or i > j2) for (i2, j2, L) in bar_levels):
                    level += 0.06
                x1, x2 = ranks_sorted[i], ranks_sorted[j]
                ax.hlines(level, min(x1, x2), max(x1, x2), lw=2.0, color="black")
                bar_levels.append((i, j, level))

    fig.tight_layout()
    fig.savefig(out_png, bbox_inches="tight")
    plt.close(fig)
    return CD

# ---------- run per metric ----------
for metric in METRICS:
    print(f"\n[metric] {metric}")
    dfm = scores.copy()

    # average ranks & Friedman
    avg_r = average_ranks_for_metric(dfm, metric)
    chi2, p_fr, N_eff = friedman_statistic(dfm, metric)   # N_eff may drop if some tasks had NaNs after pivot
    nonsig = nemenyi_nonsig_mask(avg_r, N_eff, alpha=0.05)
    CD = compute_CD(alpha=0.05, k=k, N=N_eff)

    # save average ranks (CSV + LaTeX)
    avg_csv = TABLE_DIR / f"avg_ranks_{metric}.csv"
    avg_r.to_csv(avg_csv, header=["avg_rank"])

    tex = []
    tex += [r"\begin{table}[t]",
            r"\centering",
            rf"\caption{{Average ranks over {N_eff} tasks ({metric}). Lower is better.}}",
            rf"\label{{tab:avg_ranks_{metric}}}",
            r"\small",
            r"\begin{tabular}{lc}",
            r"\toprule",
            r"\textbf{Method} & \textbf{Average rank} \\",
            r"\midrule"]
    for m, rnk in avg_r.sort_values().items():
        tex.append(f"{m} & {rnk:.2f} \\\\")
    tex += [r"\bottomrule", r"\end{tabular}", r"\end{table}"]
    with open(TABLE_DIR / f"avg_ranks_{metric}.tex", "w") as f:
        f.write("\n".join(tex))

    # save Friedman summary (LaTeX)
    sum_lines = []
    sum_lines += [r"\begin{table}[t]",
                  r"\centering",
                  rf"\caption{{Friedman--Nemenyi summary for {metric} (methods $k={k}$, tasks $N={N_eff}$).}}",
                  rf"\label{{tab:friedman_{metric}}}",
                  r"\small",
                  r"\begin{tabular}{lc}",
                  r"\toprule",
                  r"$\chi^2_{\mathrm{Friedman}}$ & " + f"{chi2:.3f} \\\\",
                  r"$p$-value & " + (f"{p_fr:.3g}" if p_fr >= 1e-6 else f"{p_fr:.2e}") + r" \\",
                  r"Critical difference (Nemenyi, $\alpha=0.05$) & " + f"{CD:.3f} \\\\",
                  r"\bottomrule",
                  r"\end{tabular}",
                  r"\end{table}"]
    with open(TABLE_DIR / f"friedman_nemenyi_{metric}.tex", "w") as f:
        f.write("\n".join(sum_lines))

    # save non-significance mask (CSV; True means "not significantly different")
    nonsig.astype(int).to_csv(TABLE_DIR / f"nemenyi_nonsig_{metric}.csv")
    print("[saved]", avg_csv.name, f"friedman_nemenyi_{metric}.tex", f"nemenyi_nonsig_{metric}.csv")

    # plot CD diagram
    out_png = TABLE_DIR / f"cd_{metric}.png"
    CD_drawn = plot_cd_diagram(avg_r, nonsig, N=N_eff, alpha=0.05,
                               out_png=out_png,
                               title=f"Critical difference diagram ({metric})")
    print("[saved]", out_png.name, f"(CD≈{CD_drawn:.2f})")

# combine three CD plots
fig, axes = plt.subplots(1, 3, figsize=(12, 2.8), dpi=200)
for ax, metric in zip(axes, METRICS):
    img = plt.imread(TABLE_DIR / f"cd_{metric}.png")
    ax.imshow(img)
    ax.axis("off")
fig.tight_layout()
combo_path = TABLE_DIR / "cd_triptych.png"
fig.savefig(combo_path, bbox_inches="tight")
plt.close(fig)
print("[saved]", combo_path)

[result dir] output/result
[loaded rows] 54
[design] N tasks = 9, k methods = 6

[metric] stability
[saved] avg_ranks_stability.csv friedman_nemenyi_stability.tex nemenyi_nonsig_stability.csv
[saved] cd_stability.png (CD≈3.55)

[metric] noise
[saved] avg_ranks_noise.csv friedman_nemenyi_noise.tex nemenyi_nonsig_noise.csv
[saved] cd_noise.png (CD≈3.55)

[metric] decoy
[saved] avg_ranks_decoy.csv friedman_nemenyi_decoy.tex nemenyi_nonsig_decoy.csv
[saved] cd_decoy.png (CD≈3.55)
[saved] output/result/tables/cd_triptych.png


In [47]:
import re
_num_re = re.compile(r"[-+]?(\d+(\.\d*)?|\.\d+)([eE][-+]?\d+)?")

def _to_float(s):
    if pd.isna(s):
        return np.nan
    m = _num_re.search(str(s))
    return float(m.group(0)) if m else np.nan

In [48]:
# --- Finder & Unifier ---
import os, shutil, pathlib, glob, pandas as pd, numpy as np, math
from typing import List

# 1) 探索候補（必要なら追加してください）
CAND_ROOTS = [
    pathlib.Path("/Users/takafumi/Documents/Python/output/HuberRidgeAIME_results"),
    pathlib.Path("/Users/takafumi/output/HuberRidgeAIME_results"),
    pathlib.Path.cwd()
]

NEEDED = [
    "avg_ranks_stability.csv","avg_ranks_noise.csv","avg_ranks_decoy.csv",
    "avg_ranks_stability.tex","avg_ranks_noise.tex","avg_ranks_decoy.tex",
    "friedman_nemenyi_stability.tex","friedman_nemenyi_noise.tex","friedman_nemenyi_decoy.tex",
    "nemenyi_nonsig_stability.csv","nemenyi_nonsig_noise.csv","nemenyi_nonsig_decoy.csv",
    "cd_stability.png","cd_noise.png","cd_decoy.png","cd_triptych.png"
]

def find_tables() -> dict:
    hit = {}
    for root in CAND_ROOTS:
        tdir = root / "tables"
        if tdir.exists():
            for p in tdir.glob("*"):
                if p.name in NEEDED:
                    hit.setdefault(p.name, []).append(p)
    return hit

hits = find_tables()
print("=== Found files by name ===")
for name in NEEDED:
    paths = hits.get(name, [])
    print(f"{name:30s} : {', '.join(str(p) for p in paths) if paths else '(not found)'}")

# 2) 統一先（ここに揃えます）
UNIFY_ROOT = pathlib.Path("/Users/takafumi/Documents/Python/output/HuberRidgeAIME_results")
UNIFY_TABLES = UNIFY_ROOT / "tables"
UNIFY_TABLES.mkdir(parents=True, exist_ok=True)

# 3) コピー（見つかったものを統一先へ）
copied = []
for name, paths in hits.items():
    # 既に統一先にあっても、更新時刻が新しい方を優先
    best = max(paths, key=lambda p: p.stat().st_mtime)
    dest = UNIFY_TABLES / name
    if not dest.exists() or best.stat().st_mtime > dest.stat().st_mtime:
        shutil.copy2(best, dest)
        copied.append(str(dest))
print("\n=== Copied to unify root ===")
print("\n".join(copied) if copied else "(nothing to copy)")

print("\nUnified tables dir:", UNIFY_TABLES)
print("ls:")
for p in sorted(UNIFY_TABLES.glob("*")):
    print(" -", p.name)

=== Found files by name ===
avg_ranks_stability.csv        : (not found)
avg_ranks_noise.csv            : (not found)
avg_ranks_decoy.csv            : (not found)
avg_ranks_stability.tex        : (not found)
avg_ranks_noise.tex            : (not found)
avg_ranks_decoy.tex            : (not found)
friedman_nemenyi_stability.tex : (not found)
friedman_nemenyi_noise.tex     : (not found)
friedman_nemenyi_decoy.tex     : (not found)
nemenyi_nonsig_stability.csv   : (not found)
nemenyi_nonsig_noise.csv       : (not found)
nemenyi_nonsig_decoy.csv       : (not found)
cd_stability.png               : (not found)
cd_noise.png                   : (not found)
cd_decoy.png                   : (not found)
cd_triptych.png                : (not found)

=== Copied to unify root ===
(nothing to copy)

Unified tables dir: /Users/takafumi/Documents/Python/output/HuberRidgeAIME_results/tables
ls:
 - overall_mean_std.tex
 - overall_mean_std_preview.csv
 - synthetic_stats.tex
 - synthetic_stats_raw.csv


In [49]:
# --- Rebuild CD assets only from existing stress_*.csv ---
import pathlib, math, numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import studentized_range

RESULT_DIR = pathlib.Path("/Users/takafumi/Documents/Python/output/HuberRidgeAIME_results")
TABLE_DIR  = RESULT_DIR / "tables"
TABLE_DIR.mkdir(parents=True, exist_ok=True)

METHODS = ["AIME","HuberAIME","RidgeAIME","HuberRidgeAIME","LIME","SHAP"]
METRICS = ["stability","noise","decoy"]

def _to_float(s): return pd.to_numeric(s, errors="coerce")

def load_scores(result_dir: pathlib.Path) -> pd.DataFrame:
    rows=[]
    for p in result_dir.glob("stress_*.csv"):
        df = pd.read_csv(p)
        df = df[["dataset","model","method","stability","noise","decoy"]]
        df["task"] = df["dataset"].astype(str)+"__"+df["model"].astype(str)
        for m in METRICS: df[m] = _to_float(df[m])
        rows.append(df)
    allsc = pd.concat(rows, ignore_index=True)
    allsc = allsc[allsc["method"].isin(METHODS)].copy()
    # 平均で重複（task×method）を畳み込み
    agg = (allsc.groupby(["task","dataset","model","method"], as_index=False)[METRICS].mean())
    return agg

def metric_matrix(df_long: pd.DataFrame, metric: str) -> pd.DataFrame:
    mat = df_long.pivot_table(index="task", columns="method", values=metric, aggfunc="mean")
    mat = mat.reindex(columns=METHODS).apply(pd.to_numeric, errors="coerce").dropna(axis=0, how="any")
    return mat

def average_ranks(df_long: pd.DataFrame, metric: str) -> pd.Series:
    mat = metric_matrix(df_long, metric)
    ranks = mat.rank(axis=1, ascending=False, method="average")
    return ranks.mean(axis=0).reindex(METHODS)

def friedman(df_long: pd.DataFrame, metric: str):
    mat = metric_matrix(df_long, metric)
    groups = [mat[m].values for m in METHODS]
    chi2, p = stats.friedmanchisquare(*groups)
    return float(chi2), float(p), mat.shape[0]  # N_eff

def compute_CD(alpha: float, k: int, N: int) -> float:
    q = studentized_range.isf(alpha, k, np.inf)
    return float(q * math.sqrt(k*(k+1)/(6.0*N)))

def nonsig_mask(avg_ranks: pd.Series, N: int, alpha=0.05) -> pd.DataFrame:
    CD = compute_CD(alpha, len(avg_ranks), N)
    idx = avg_ranks.index
    M = pd.DataFrame(False, index=idx, columns=idx)
    for a in idx:
        for b in idx:
            M.loc[a,b] = (a==b) or (abs(avg_ranks[a]-avg_ranks[b]) <= CD)
    return M

def plot_cd(avg_r, nonsig, N, alpha, out_png, title):
    k=len(avg_r); CD=compute_CD(alpha,k,N)
    order=np.argsort(avg_r.values)
    ms=avg_r.index.values[order]; rs=avg_r.values[order]
    fig,ax=plt.subplots(figsize=(8,2.2),dpi=200)
    ax.set_title(title,fontsize=10); ax.set_xlim(1-0.2,k+0.2); ax.set_ylim(0,1); ax.get_yaxis().set_visible(False)
    ax.hlines(0.5,1,k,lw=1,color="black")
    for r in range(1,k+1):
        ax.vlines(r,0.47,0.53,color="black",lw=1); ax.text(r,0.56,f"{r}",ha="center",va="bottom",fontsize=8)
    ax.hlines(0.9,1,1+CD,lw=2,color="black"); ax.vlines(1,0.88,0.92,lw=1.5,color="black"); ax.vlines(1+CD,0.88,0.92,lw=1.5,color="black")
    ax.text(1+CD/2,0.93,f"CD={CD:.2f}",ha="center",va="bottom",fontsize=8)
    y0=0.4
    for i,(m,r) in enumerate(zip(ms,rs)):
        ax.plot([r],[y0],"o",ms=4,color="black")
        ax.text(r, y0+(0.10 if i%2==0 else -0.12), m, ha="center", va="center", fontsize=8)
    # non-significant cliques
    def all_nonsig(i,j):
        idx=ms[i:j+1]
        return all(nonsig.loc[a,b] for x,a in enumerate(idx) for b in idx[x+1:])
    bars=[]
    for span in range(1,k):
        for i in range(0,k-span):
            j=i+span
            if all_nonsig(i,j):
                level=0.72
                while any(abs(level-L)<0.06 and not (j<i2 or i>j2) for (i2,j2,L) in bars):
                    level+=0.06
                x1,x2=rs[i],rs[j]
                ax.hlines(level, min(x1,x2), max(x1,x2), lw=2, color="black")
                bars.append((i,j,level))
    fig.tight_layout(); fig.savefig(out_png, bbox_inches="tight"); plt.close(fig)

scores = load_scores(RESULT_DIR)
for metric in METRICS:
    avg = average_ranks(scores, metric)
    chi2,p,N_eff = friedman(scores, metric)
    ns = nonsig_mask(avg, N_eff, alpha=0.05)
    # 保存
    (TABLE_DIR / f"avg_ranks_{metric}.csv").write_text(avg.to_csv(header=["avg_rank"]))
    with open(TABLE_DIR / f"avg_ranks_{metric}.tex","w") as f:
        f.write("\n".join([
            r"\begin{table}[t]", r"\centering",
            rf"\caption{{Average ranks over {N_eff} tasks ({metric}). Lower is better.}}",
            rf"\label{{tab:avg_ranks_{metric}}}", r"\small",
            r"\begin{tabular}{lc}", r"\toprule",
            r"\textbf{Method} & \textbf{Average rank} \\",
            r"\midrule", *[f"{m} & {r:.2f} \\\\" for m,r in avg.sort_values().items()],
            r"\bottomrule", r"\end{tabular}", r"\end{table}"
        ]))
    ns.astype(int).to_csv(TABLE_DIR / f"nemenyi_nonsig_{metric}.csv")
    with open(TABLE_DIR / f"friedman_nemenyi_{metric}.tex","w") as f:
        f.write("\n".join([
            r"\begin{table}[t]", r"\centering",
            rf"\caption{{Friedman--Nemenyi summary for {metric} (methods $k={len(METHODS)}$, tasks $N={N_eff}$).}}",
            rf"\label{{tab:friedman_{metric}}}", r"\small",
            r"\begin{tabular}{lc}", r"\toprule",
            r"$\chi^2_{\mathrm{Friedman}}$ & " + f"{chi2:.3f} \\\\",
            r"$p$-value & " + (f"{p:.3g}" if p>=1e-6 else f"{p:.2e}") + r" \\",
            r"Critical difference (Nemenyi, $\alpha=0.05$) & " + f"{compute_CD(0.05, len(METHODS), N_eff):.3f} \\\\",
            r"\bottomrule", r"\end{tabular}", r"\end{table}"
        ]))
    plot_cd(avg, ns, N_eff, 0.05, TABLE_DIR / f"cd_{metric}.png",
            title=f"Critical difference diagram ({metric})")

# 3枚を横並びの合成
fig,axes=plt.subplots(1,3,figsize=(12,2.8),dpi=200)
for ax,metric in zip(axes, METRICS):
    ax.imshow(plt.imread(TABLE_DIR / f"cd_{metric}.png")); ax.axis("off")
fig.tight_layout(); fig.savefig(TABLE_DIR / "cd_triptych.png", bbox_inches="tight"); plt.close(fig)

print("Wrote to:", TABLE_DIR)
for p in sorted(TABLE_DIR.glob("*")): print(" -", p.name)

Wrote to: /Users/takafumi/Documents/Python/output/HuberRidgeAIME_results/tables
 - avg_ranks_decoy.csv
 - avg_ranks_decoy.tex
 - avg_ranks_noise.csv
 - avg_ranks_noise.tex
 - avg_ranks_stability.csv
 - avg_ranks_stability.tex
 - cd_decoy.png
 - cd_noise.png
 - cd_stability.png
 - cd_triptych.png
 - friedman_nemenyi_decoy.tex
 - friedman_nemenyi_noise.tex
 - friedman_nemenyi_stability.tex
 - nemenyi_nonsig_decoy.csv
 - nemenyi_nonsig_noise.csv
 - nemenyi_nonsig_stability.csv
 - overall_mean_std.tex
 - overall_mean_std_preview.csv
 - synthetic_stats.tex
 - synthetic_stats_raw.csv


In [52]:
# turn frontier_wdbc_svm.csv into a figure
import pandas as pd, matplotlib.pyplot as plt, pathlib
df = pd.read_csv("../output/frontier_wdbc_svm.csv")  # 生成先に合わせてパス調整
out = pathlib.Path("../output/frontier_wdbc_svm.png")
fig, ax = plt.subplots(figsize=(4.8,3), dpi=200)
for m, g in df.groupby("method"):
    g = g.sort_values("budget")
    ax.plot(g["budget"], g["noise"], marker="o", label=m)  # yを 'stability' に変える案も可
ax.set_xscale("log")
ax.set_xlabel("Sampling budget (log-scale)")
ax.set_ylabel("Noise robustness")
ax.legend(frameon=False)
fig.tight_layout(); fig.savefig(out, bbox_inches="tight"); plt.close(fig)
print("saved:", out)

saved: ../output/frontier_wdbc_svm.png


In [53]:
import pathlib
need = [
    "tables/overall_mean_std_clean.tex",
    "tables/synthetic_paired_with_ci.tex",
    "tables/cd_stability.png", "tables/cd_noise.png", "tables/cd_decoy.png",
    "tables/avg_ranks_stability.tex", "tables/avg_ranks_noise.tex", "tables/avg_ranks_decoy.tex",
    "tables/friedman_nemenyi_stability.tex", "tables/friedman_nemenyi_noise.tex", "tables/friedman_nemenyi_decoy.tex",
    "tables/hra_consistency.tex",
    "tables/frontier_wdbc_svm.png",
]
root = pathlib.Path("tables")  # 生成ディレクトリに合わせて調整
missing = [p for p in need if not pathlib.Path(p).exists()]
print("MISSING:" if missing else "OK (all found)")
for m in missing: print(" -", m)

MISSING:
 - tables/overall_mean_std_clean.tex
 - tables/synthetic_paired_with_ci.tex
 - tables/cd_stability.png
 - tables/cd_noise.png
 - tables/cd_decoy.png
 - tables/avg_ranks_stability.tex
 - tables/avg_ranks_noise.tex
 - tables/avg_ranks_decoy.tex
 - tables/friedman_nemenyi_stability.tex
 - tables/friedman_nemenyi_noise.tex
 - tables/friedman_nemenyi_decoy.tex
 - tables/hra_consistency.tex
 - tables/frontier_wdbc_svm.png
